In [ ]:
!pip install rut_chile
!pip install levenshtein
!pip install Unidecode
from IPython.display import clear_output
clear_output()

In [ ]:
import os
import requests
import pandas as pd
import concurrent.futures
from tqdm import tqdm

from rut_chile import rut_chile
from unidecode import unidecode
from Levenshtein import distance

## Cargar datos

In [ ]:
data_dir = "/content/drive/Shareddrives/Estudio Ecosistema CTCI/PT2_Sistematización fuentes de información/Análisis/ANID_CORFO/Datos/ANID"
results_dir = "/content/drive/Shareddrives/Estudio Ecosistema CTCI/PT2_Sistematización fuentes de información/Análisis/ANID_CORFO/Resultados/ANID"

In [ ]:
df_scia = pd.read_excel(os.path.join(data_dir, "SCIA_INSTITUCIONES.xlsx"))
df_sia = pd.read_excel(os.path.join(data_dir, "SIA_INSTITUCIONES.xlsx"))
df_spi = pd.read_excel(os.path.join(data_dir, "SPI_INSTITUCIONES.xlsx"))
df_srec = pd.read_excel(os.path.join(data_dir, "SREC_INSTITUCIONES.xlsx"))


In [ ]:
df_scia["SUBDIRECCION"] = "SCIA"
df_sia["SUBDIRECCION"] = "SIA"
df_spi["SUBDIRECCION"] = "SPI"
df_srec["SUBDIRECCION"] = "SREC"

df_anid = pd.concat([df_scia, df_sia, df_spi, df_srec], ignore_index=True)
df_anid

,PROGRAMA,CONCURSO,FOLIO,AGNOPOSTULACION,AGNOFALLO,RESPONSABLEPROYECTO,SEXO,ROLINSTITUCION,INSTITUCION1ERNIVEL,RUTINSTITUCION,PAISINSTITUCION,REGIONINSTITUCION,SUBDIRECCION
0,REGIONAL,II CONCURSO DE FORTALECIMIENTO A LA CONTINUIDA...,R15A10001,2015,2016,EDUARDO GRATACOS,HOMBRE,BENEFICIARIA PRINCIPAL,CENTRO REGIONAL DE INNOVACION HORTOFRUTICOLA ...,SIN INFORMACION,CHILE,REGION DE VALPARAISO,SCIA
1,REGIONAL,II CONCURSO DE FORTALECIMIENTO A LA CONTINUIDA...,R15A10002,2015,2016,LUIS ROJAS ARAYA,HOMBRE,BENEFICIARIA PRINCIPAL,CENTRO DE INVESTIGACION CIENTIFICO TECNOLOGICO...,SIN INFORMACION,CHILE,REGION DE ANTOFAGASTA,SCIA
2,PIA,CONCURSO NACIONAL DE ASIGNACION DE TIEMPO DE B...,AUB160003,2016,2016,RUBEN ESCRIBANO VELOSO,HOMBRE,INSTITUCION PATROCINANTE,UNIVERSIDAD DE CONCEPCION,81494400-K,CHILE,REGION DEL BIO-BIO,SCIA
3,REGIONAL,II CONCURSO DE FORTALECIMIENTO DE CENTROS REGI...,R16F10008,2016,2016,EDUARDO GRATACOS N.,HOMBRE,BENEFICIARIA PRINCIPAL,CENTRO REGIONAL DE INNOVACION HORTOFRUTICOLA ...,SIN INFORMACION,CHILE,REGION DE VALPARAISO,SCIA
4,REGIONAL,II CONCURSO DE FORTALECIMIENTO DE CENTROS REGI...,R16F10009,2016,2016,BERNARDO BROITMAN ROJAS,HOMBRE,BENEFICIARIA PRINCIPAL,CENTRO DE ESTUDIOS AVANZADOS EN ZONAS ARIDAS (...,SIN INFORMACION,CHILE,REGION DE COQUIMBO,SCIA
...,...,...,...,...,...,...,...,...,...,...,...,...,...
23703,PCI,PROGRAMAS REGIONALES STIC-AMSUD - MATH-AMSUD -...,AMSUD220044,2022,2023,DENISSE MUÑANTE ARZAPALO,FEMENINO,INSTITUCION DE FRANCIA,ECOLE NATIONALE SUPERIEURE D’INFORMATIQUE POUR...,NaN,FRANCIA,NaN,SREC
23704,PCI,PROGRAMAS REGIONALES STIC-AMSUD - MATH-AMSUD -...,AMSUD220045,2022,2023,HUGO CARRILLO LINCOPI,MASCULINO,INSTITUCION DE FRANCIA,INRIA PARIS,NaN,FRANCIA,NaN,SREC
23705,PCI,PROGRAMAS REGIONALES STIC-AMSUD - MATH-AMSUD -...,AMSUD220045,2022,2023,HUGO CARRILLO LINCOPI,MASCULINO,INSTITUCION SUDAMERICANA,ESCUELA POLITECNICA NACIONAL,NaN,ECUADOR,NaN,SREC
23706,PCI,PROGRAMAS REGIONALES STIC-AMSUD - MATH-AMSUD -...,AMSUD220045,2022,2023,HUGO CARRILLO LINCOPI,MASCULINO,INSTITUCION SUDAMERICANA,FUNDACION INRIA CHILE,65058044-3,CHILE,NaN,SREC


## Verificar ruts válidos

In [ ]:
def valid_rut_wrapper(input):
  if type(input) != str:
    return False
  elif not (input[0].isdigit() and (input[-1].isdigit() or input[-1].lower() == "k")):
    return False
  else:
    return rut_chile.is_valid_rut(input)

df_anid["rut_normalizado"] = df_anid["RUTINSTITUCION"].str.lower().str.strip().str.replace(" ", "")
df_anid["rut_valido"] = df_anid["rut_normalizado"].apply(valid_rut_wrapper)

In [ ]:
df_anid.loc[~df_anid["rut_valido"], "rut_normalizado"].unique()

array(['sininformacion', 'noaplica', '-', '65117438-k', '619598008-',
       '65343160-', '64146448-k', '69160700-', '604511851-', '62336893-5',
       '60701000-1', '75751100-8', '76399541-1', '72323600-k', '818002-1',
       '87674400-k', '7972158-5', '61101000-k', '61606201-', '60908000-',
       nan, '87674400-', '65182552-3'], dtype=object)

## Normalizar nombres

In [ ]:
df_anid["nombre_normalizado"] = df_anid["INSTITUCION1ERNIVEL"].str.upper().str.strip().str.replace(".", "").str.replace("  ", " ").apply(unidecode)

<ipython-input-8-aa24c799fc9e>:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df_anid["nombre_normalizado"] = df_anid["INSTITUCION1ERNIVEL"].str.upper().str.strip().str.replace(".", "").str.replace("  ", " ").apply(unidecode)


## Arreglar RUTs



Veamos primero cuántas filas tienen RUTs inválidos (deben ser instituciones chilenas)

In [ ]:
df_anid[(~df_anid["rut_valido"]) & (df_anid["PAISINSTITUCION"] == "CHILE")]

,PROGRAMA,CONCURSO,FOLIO,AGNOPOSTULACION,AGNOFALLO,RESPONSABLEPROYECTO,SEXO,ROLINSTITUCION,INSTITUCION1ERNIVEL,RUTINSTITUCION,PAISINSTITUCION,REGIONINSTITUCION,SUBDIRECCION,rut_normalizado,rut_valido,nombre_normalizado
0,REGIONAL,II CONCURSO DE FORTALECIMIENTO A LA CONTINUIDA...,R15A10001,2015,2016,EDUARDO GRATACOS,HOMBRE,BENEFICIARIA PRINCIPAL,CENTRO REGIONAL DE INNOVACION HORTOFRUTICOLA ...,SIN INFORMACION,CHILE,REGION DE VALPARAISO,SCIA,sininformacion,False,CENTRO REGIONAL DE INNOVACION HORTOFRUTICOLA D...
1,REGIONAL,II CONCURSO DE FORTALECIMIENTO A LA CONTINUIDA...,R15A10002,2015,2016,LUIS ROJAS ARAYA,HOMBRE,BENEFICIARIA PRINCIPAL,CENTRO DE INVESTIGACION CIENTIFICO TECNOLOGICO...,SIN INFORMACION,CHILE,REGION DE ANTOFAGASTA,SCIA,sininformacion,False,CENTRO DE INVESTIGACION CIENTIFICO TECNOLOGICO...
3,REGIONAL,II CONCURSO DE FORTALECIMIENTO DE CENTROS REGI...,R16F10008,2016,2016,EDUARDO GRATACOS N.,HOMBRE,BENEFICIARIA PRINCIPAL,CENTRO REGIONAL DE INNOVACION HORTOFRUTICOLA ...,SIN INFORMACION,CHILE,REGION DE VALPARAISO,SCIA,sininformacion,False,CENTRO REGIONAL DE INNOVACION HORTOFRUTICOLA D...
4,REGIONAL,II CONCURSO DE FORTALECIMIENTO DE CENTROS REGI...,R16F10009,2016,2016,BERNARDO BROITMAN ROJAS,HOMBRE,BENEFICIARIA PRINCIPAL,CENTRO DE ESTUDIOS AVANZADOS EN ZONAS ARIDAS (...,SIN INFORMACION,CHILE,REGION DE COQUIMBO,SCIA,sininformacion,False,CENTRO DE ESTUDIOS AVANZADOS EN ZONAS ARIDAS (...
5,REGIONAL,II CONCURSO DE FORTALECIMIENTO DE CENTROS REGI...,R16F10002,2016,2016,CESAR BURGOS,HOMBRE,BENEFICIARIA PRINCIPAL,CENTRO DE GENOMICA NUTRICIONAL AGROACUICOLA (C...,SIN INFORMACION,CHILE,REGION DE LA ARAUCANIA,SCIA,sininformacion,False,CENTRO DE GENOMICA NUTRICIONAL AGROACUICOLA (C...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22954,PCI,FOMENTO A LA VINCULACION INTERNACIONAL PARA IN...,FOVI220124,2022,2022,MARISOL RUIZ CONTRERAS,MUJER,INSTITUCION ASOCIADA,ILUSTRE MUNICIPALIDAD DE VALDIVIA,-,CHILE,REGION DE LOS RIOS,SREC,-,False,ILUSTRE MUNICIPALIDAD DE VALDIVIA
22955,PCI,FOMENTO A LA VINCULACION INTERNACIONAL PARA IN...,FOVI220211,2022,2022,DANIEL MEDINA SALAS,HOMBRE,INSTITUCION ASOCIADA,SEREMI DE SALUD REGION DE LOS LAGOS,-,CHILE,NO REGISTRA REGION,SREC,-,False,SEREMI DE SALUD REGION DE LOS LAGOS
22956,PCI,FOMENTO A LA VINCULACION INTERNACIONAL PARA IN...,FOVI220211,2022,2022,DANIEL MEDINA SALAS,HOMBRE,INSTITUCION ASOCIADA,INSTITUTO NORUEGO DE INVESTIGACION DE AGUAS,-,CHILE,NO REGISTRA REGION,SREC,-,False,INSTITUTO NORUEGO DE INVESTIGACION DE AGUAS
23173,PCI,FOMENTO A LA VINCULACION INTERNACIONAL PARA IN...,FOVI220135,2022,2022,ARIEL LEIVA LOPEZ,HOMBRE,INSTITUCION EXTRANJERA,COMISION ECONOMICA PARA AMERICA LATINA Y EL CA...,NO APLICA,CHILE,REGION METROPOLITANA DE SANTIAGO,SREC,noaplica,False,COMISION ECONOMICA PARA AMERICA LATINA Y EL CA...


916 filas.

Ok, lo primero que haremos será normalizar un poco más los RUTS. Para ello, cambiaremos todos los ruts "sininformacion" a "-"

In [ ]:
df_anid.loc[df_anid["rut_normalizado"] == "sininformacion", "rut_normalizado"] = "-"

### Arreglo manual de RUTs con errores de tipeo

Fijémonos ahora en cuántas instituciones tienen RUTs que parecen ser válidos, pero no lo son.

In [ ]:
df_anid[(~df_anid["rut_normalizado"].isnull())
          & (~df_anid["rut_valido"])
          & (df_anid["PAISINSTITUCION"] == "CHILE")
          & (~df_anid["rut_normalizado"].isin(["-", "noaplica"]))]

,PROGRAMA,CONCURSO,FOLIO,AGNOPOSTULACION,AGNOFALLO,RESPONSABLEPROYECTO,SEXO,ROLINSTITUCION,INSTITUCION1ERNIVEL,RUTINSTITUCION,PAISINSTITUCION,REGIONINSTITUCION,SUBDIRECCION,rut_normalizado,rut_valido,nombre_normalizado
1682,FONDEF,V CONCURSO DE INVESTIGACION TECNOLOGICA DEL FO...,IT16I10096,2016,2017,ROBERTO ALDUNATE VERA,HOMBRE,BENEFICIARIA PRINCIPAL,CENTRO DE INNOVACION E INVENTIGACION APLICADA,65117438-K,CHILE,REGION DE LOS LAGOS,SIA,65117438-k,False,CENTRO DE INNOVACION E INVENTIGACION APLICADA
2308,FONDEF,PRIMER CONCURSO IDEA EN DOS ETAPAS TEMATICO EN...,ID16AM2080,2018,2019,EDUARDO TOBAR ALMONACID,HOMBRE,ENTIDAD ASOCIADA,"HOSPITAL SANTIAGO ORIENTE ""DR. LUIS TISNE BROU...",619598008-,CHILE,NO REGISTRA REGION,SIA,619598008-,False,"HOSPITAL SANTIAGO ORIENTE ""DR LUIS TISNE BROUSSE"""
2311,FONDEF,III CONCURSO IDEA EN DOS ETAPAS . SEGUNDA ETAPA,ID16I20013,2018,2019,MARIA ANGELICA LARRAIN BARTH,MUJER,ENTIDAD ASOCIADA,ASOCIACION GREMIAL DE MITILICULTORES DE CHILE,65343160-,CHILE,REGION DE LOS LAGOS,SIA,65343160-,False,ASOCIACION GREMIAL DE MITILICULTORES DE CHILE
2731,FONIS,XV CONCURSO NACIONAL DE PROYECTOS DE INVESTIGA...,SA18I0116,2018,2018,VALERIA CAMPOS CANNOBBIO,MUJER,ENTIDAD ASOCIADA,FUNDACION KIMNTRUM,64146448-K,CHILE,REGION DEL BIO-BIO,SIA,64146448-k,False,FUNDACION KIMNTRUM
3691,FONDEF,CONCURSO IDEA I+D 2020,ID20I10418,2020,2020,ROBERTO ALDUNATE VERA,HOMBRE,BENEFICIARIA PRINCIPAL,CENTRO DE INNOVACION E INVENTIGACION APLICADA,65117438-K,CHILE,REGION DE LOS LAGOS,SIA,65117438-k,False,CENTRO DE INNOVACION E INVENTIGACION APLICADA
3771,FONDEF,IX CONCURSO INVESTIGACION TECNOLOGICA – IDEA 2020,IT20I0002,2020,2021,RAQUEL MONTESINO SEGUI,MUJER,ENTIDAD ASOCIADA / MANDANTE,CENTRO DE DIAGNOSTICO VETERINARIO S.A.,604511851-,CHILE,NO REGISTRA REGION,SIA,604511851-,False,CENTRO DE DIAGNOSTICO VETERINARIO SA
3772,FONDEF,IX CONCURSO INVESTIGACION TECNOLOGICA – IDEA 2020,IT20I0002,2020,2021,RAQUEL MONTESINO SEGUI,MUJER,ENTIDAD ASOCIADA / MANDANTE,VETANCO S.A.,62336893-5,CHILE,NO REGISTRA REGION,SIA,62336893-5,False,VETANCO SA
3815,FONDEF,IX CONCURSO INVESTIGACION TECNOLOGICA – IDEA 2020,IT20I0028,2020,2021,HUMBERTO PALZA CORDERO,HOMBRE,ENTIDAD ASOCIADA / MANDANTE,CENTRO DE ENVASES Y EMBALAJES DE CHILE CENEM,75751100-8,CHILE,REGION METROPOLITANA DE SANTIAGO,SIA,75751100-8,False,CENTRO DE ENVASES Y EMBALAJES DE CHILE CENEM
3848,FONDEF,IX CONCURSO INVESTIGACION TECNOLOGICA – IDEA 2020,IT20I0084,2020,2021,JUAN MUNIZAGA PLAZA,HOMBRE,ENTIDAD ASOCIADA / MANDANTE,SERVICIOS INTEGRADOS MENVAL LIMITADA,76399541-1,CHILE,REGION DE COQUIMBO,SIA,76399541-1,False,SERVICIOS INTEGRADOS MENVAL LIMITADA
3942,FONDEF,CONCURSO IDEA I+D 2021,ID21I10061,2021,2021,SYLVIA CONTRERAS SALINAS,MUJER,ENTIDAD ASOCIADA,FUNDACION EDUCACIONAL SUMATE PADRE ALVARO LAVIN,72323600-K,CHILE,REGION METROPOLITANA DE SANTIAGO,SIA,72323600-k,False,FUNDACION EDUCACIONAL SUMATE PADRE ALVARO LAVIN


Varios de estos casos son errores de tipeo (en el dígito verificador o faltó un número entremedio). Los arreglaremos

In [ ]:
to_fix = [("CENTRO DE INNOVACION E INVENTIGACION APLICADA", "65117438-4"),
          ("AGRICOLA LOS OLMOS LTDA", "79721580-5"),
          ("HOSPITAL REGIONAL DE ANTOFAGASTA DR LEONARDO GUZMAN", "61606201-8"),
          ("JUNTA NACIONAL DE AUXILIO ESCOLAR Y BECAS, PROGRAMA HABILIDADES PARA LA VIDA", "60908000-0"),
          ('HOSPITAL SANTIAGO ORIENTE "DR LUIS TISNE BROUSSE"', '61959800-8'),
          ('ASOCIACION GREMIAL DE MITILICULTORES DE CHILE', '65343160-0'),
          ('FUNDACION EDUCACIONAL SUMATE PADRE ALVARO LAVIN', '72323600-2'),
          ('LABORATORIO PASTEUR SA', '87674400-7'),
          ('CENTRO REGIONAL DE INVESTIGACION E INNOVACION PARA LA SOSTENIBILIDAD DE LA AGRICULTURA Y LOS TERRITORIOS RURALES', '65182552-0')]
for (name, rut) in to_fix:
  df_anid.loc[df_anid["nombre_normalizado"] == name,
              ["rut_normalizado", "rut_valido"]] = (rut, True)

df_anid[df_anid["nombre_normalizado"].isin([name for name, rut in to_fix])]


,PROGRAMA,CONCURSO,FOLIO,AGNOPOSTULACION,AGNOFALLO,RESPONSABLEPROYECTO,SEXO,ROLINSTITUCION,INSTITUCION1ERNIVEL,RUTINSTITUCION,PAISINSTITUCION,REGIONINSTITUCION,SUBDIRECCION,rut_normalizado,rut_valido,nombre_normalizado
163,FONDEQUIP,VI CONCURSO DE EQUIPAMIENTO CIENTIFICO Y TECNO...,EQM170023,2017,2017,CLAUDIA MARDONES PEÃ±A,MUJER,INSTITUCION ASOCIADA,LABORATORIO PASTEUR S.A.,87674400-7,CHILE,REGION DEL BIO-BIO,SCIA,87674400-7,True,LABORATORIO PASTEUR SA
1395,REGIONAL,VII CONCURSO DE FORTALECIMIENTO Y DESARROLLO D...,R22A0002,2022,2022,CARLOS HUENCHULEO PEDREROS,HOMBRE,INSTITUCION BENEFICIARIA,CENTRO REGIONAL DE INVESTIGACION E INNOVACION ...,65182552-0,CHILE,REGION DE VALPARAISO,SCIA,65182552-0,True,CENTRO REGIONAL DE INVESTIGACION E INNOVACION ...
1496,FONDEF,PRIMER CONCURSO IDEA EN DOS ETAPAS TEMATICO EN...,ID16AM0007,2016,2016,MARIA SEPULVEDA CARRENO,MUJER,INTERESADA,LABORATORIO PASTEUR S.A.,87674400-7,CHILE,REGION DEL BIO-BIO,SIA,87674400-7,True,LABORATORIO PASTEUR SA
1682,FONDEF,V CONCURSO DE INVESTIGACION TECNOLOGICA DEL FO...,IT16I10096,2016,2017,ROBERTO ALDUNATE VERA,HOMBRE,BENEFICIARIA PRINCIPAL,CENTRO DE INNOVACION E INVENTIGACION APLICADA,65117438-K,CHILE,REGION DE LOS LAGOS,SIA,65117438-4,True,CENTRO DE INNOVACION E INVENTIGACION APLICADA
1855,FONDEF,PRIMER CONCURSO IDEA FONDEF EN DOS ETAPAS - SE...,ID14I20084,2017,2017,MARIA BARRIA CARCAMO,MUJER,ENTIDAD ASOCIADA,LABORATORIO PASTEUR S.A.,87674400-7,CHILE,REGION DEL BIO-BIO,SIA,87674400-7,True,LABORATORIO PASTEUR SA
2056,FONDEF,IV CONCURSO IDEA EN DOS ETAPAS CONVOCATORIA 2017,ID17I10143,2017,2017,PABLO GONZALEZ MUNOZ,HOMBRE,INTERESADA / MANDANTE,LABORATORIO PASTEUR S.A.,87674400-7,CHILE,REGION DEL BIO-BIO,SIA,87674400-7,True,LABORATORIO PASTEUR SA
2160,FONDEF,SEXTO CONCURSO INVESTIGACION TECNOLOGICA. IDEA...,IT17I0048,2017,2018,OLIBERTO SANCHEZ RAMOS,HOMBRE,ENTIDAD ASOCIADA,LABORATORIO PASTEUR S.A.,87674400-7,CHILE,REGION DEL BIO-BIO,SIA,87674400-7,True,LABORATORIO PASTEUR SA
2308,FONDEF,PRIMER CONCURSO IDEA EN DOS ETAPAS TEMATICO EN...,ID16AM2080,2018,2019,EDUARDO TOBAR ALMONACID,HOMBRE,ENTIDAD ASOCIADA,"HOSPITAL SANTIAGO ORIENTE ""DR. LUIS TISNE BROU...",619598008-,CHILE,NO REGISTRA REGION,SIA,61959800-8,True,"HOSPITAL SANTIAGO ORIENTE ""DR LUIS TISNE BROUSSE"""
2311,FONDEF,III CONCURSO IDEA EN DOS ETAPAS . SEGUNDA ETAPA,ID16I20013,2018,2019,MARIA ANGELICA LARRAIN BARTH,MUJER,ENTIDAD ASOCIADA,ASOCIACION GREMIAL DE MITILICULTORES DE CHILE,65343160-,CHILE,REGION DE LOS LAGOS,SIA,65343160-0,True,ASOCIACION GREMIAL DE MITILICULTORES DE CHILE
2328,FONDEF,III CONCURSO IDEA EN DOS ETAPAS . SEGUNDA ETAPA,ID16I20140,2018,2020,ROBERTO VIDAL ALVAREZ,HOMBRE,ENTIDAD ASOCIADA,LABORATORIO PASTEUR S.A.,87674400-7,CHILE,REGION DEL BIO-BIO,SIA,87674400-7,True,LABORATORIO PASTEUR SA


Para el resto de las instituciones con RUT que parecen ser válidos, pero no lo son, pondremos el valor "-".

In [ ]:
df_anid.loc[(~df_anid["rut_normalizado"].isnull())
          & (~df_anid["rut_valido"])
          & (df_anid["PAISINSTITUCION"] == "CHILE")
          & (~df_anid["rut_normalizado"].isin(["-", "noaplica"])),
        "rut_normalizado"] = "-"

### Sólo un RUT por nombre de institución

Ahora, impondremos que cada institución sólo ṕuede tener un RUT. Veamos cuántos casos no cumplen con esa condición.

In [ ]:
for name in df_anid.loc[df_anid["PAISINSTITUCION"] == "CHILE", "nombre_normalizado"].unique():
  ruts = df_anid.loc[df_anid["nombre_normalizado"] == name, "rut_normalizado"].unique()
  if len(ruts) != 1:
    print(name, ruts, len(ruts))

UNIVERSIDAD DE CONCEPCION ['81494400-k' '-'] 2
JUNTA NACIONAL DE AUXILIO ESCOLAR Y BECAS ['60908000-0' '-'] 2
PONTIFICIA UNIVERSIDAD CATOLICA DE CHILE ['81698900-0' '-'] 2
UNIVERSIDAD DE LA FRONTERA ['87912900-1' '-'] 2
UNIVERSIDAD CATOLICA DEL NORTE ['81518400-9' '-'] 2
UNIVERSIDAD DE LOS LAGOS ['70772100-6' '-'] 2
UNIVERSIDAD DE TALCA ['70885500-6' '-'] 2
CORPORACION NACIONAL FORESTAL ['61313000-4' '-'] 2
INSTITUTO DE INVESTIGACIONES AGROPECUARIAS ['61312000-9' '-'] 2
SERVICIO NACIONAL DE PESCA ['60701004-8' '-'] 2
SERVICIO NACIONAL DE TURISMO ['-' '60704000-1'] 2
INSTITUTO DE NEUROCIENCIA BIOMEDICA ['65059721-4' '-'] 2
SERVICIO AGRICOLA Y GANADERO ['61308000-7' '-'] 2
VINA CONCHA Y TORO ['-' '90227000-0'] 2
UNIVERSIDAD ALBERTO HURTADO ['73923400-k' '-'] 2
MINISTERIO DE OBRAS PUBLICAS TRANSPORTES Y TELECOMUNICACIONES ['-' '61202000-0'] 2
UNIVERSIDAD ADOLFO IBANEZ ['71543200-5' '-'] 2
BIO INSUMOS NATIVA SPA ['-' '77807250-5'] 2
EMPRESA DE SERVICIOS TECNOLOGICOS ['77029400-2' '-'] 2
UN

Son varios. Fijémonos primero en los casos como el de la Universidad de Concepción, que tiene un RUT válido y otro rut inexistente.

#### Instituciones con un RUT válido y otro RUT inválido

In [ ]:
names_without_rut = set(df_anid[(~df_anid["rut_valido"]) & (df_anid["PAISINSTITUCION"] == "CHILE")]["nombre_normalizado"].unique())
names_with_rut = set(df_anid[(df_anid["rut_valido"]) & (df_anid["PAISINSTITUCION"] == "CHILE")]["nombre_normalizado"].unique())

names_intersection = names_without_rut & names_with_rut
sorted(names_intersection)

['AGROSUPER SA',
 'AGUAS ANDINAS',
 'ASOCIACION CHILENA DE MUNICIPALIDADES',
 'BIO INSUMOS NATIVA SPA',
 'CELLS FOR CELLS SA',
 'CLINICA LAS CONDES',
 'COLORADO CHILE SA',
 'COMISION NACIONAL DE ENERGIA',
 'COMPANIA SIDERURGICA HUACHIPATO SA',
 'CORPORACION CATIM',
 'CORPORACION MUNICIPAL DE LA FLORIDA',
 'CORPORACION NACIONAL FORESTAL',
 'EMPRESA DE SERVICIOS TECNOLOGICOS',
 'FEDERACION DE PRODUCTORES DE FRUTAS DE CHILE',
 'FORESTAL ARAUCO LTDA',
 'FUNDACION KIMNTRUM',
 'FUNDACION TECNOLOGICA PARA LA MINERIA',
 'GENDARMERIA DE CHILE',
 'GOBIERNO REGIONAL DE LOS LAGOS',
 'INSTITUTO DE INVESTIGACIONES AGROPECUARIAS',
 'INSTITUTO DE NEUROCIENCIA BIOMEDICA',
 'INSTITUTO DEL CEMENTO Y DEL HORMIGON DE CHILE',
 'JUNTA NACIONAL DE AUXILIO ESCOLAR Y BECAS',
 'MINISTERIO DE EDUCACION',
 'MINISTERIO DE LA VIVIENDA Y URBANISMO',
 'MINISTERIO DE OBRAS PUBLICAS TRANSPORTES Y TELECOMUNICACIONES',
 'MUNICIPALIDAD DE PUCON',
 'PONTIFICIA UNIVERSIDAD CATOLICA DE CHILE',
 'SALMONES ANTARTICA SA',
 'SERV

Hay varias instituciones con un RUT válido y otro inválido. Para esos casos, en las filas con RUT inválido, imputaremos el RUT válido asociado con la misma institución, proveniente de otra fila.

In [ ]:
for name in names_intersection:
  ruts = df_anid.loc[(df_anid["nombre_normalizado"] == name) & (df_anid["rut_valido"]),
              "rut_normalizado"].unique()
  assert len(ruts) == 1
  rut = ruts[0]
  df_anid.loc[(df_anid["nombre_normalizado"] == name) & (~df_anid["rut_valido"]),
              ("rut_valido", "rut_normalizado")] = (True, rut)

Revisaremos que haya funcionado, seleccionando alguna de las instituciones que estaban en esta situación.

In [ ]:
print(df_anid.loc[(df_anid["nombre_normalizado"] == "UNIVERSIDAD ALBERTO HURTADO"), "rut_normalizado"].unique())
print(df_anid.loc[(df_anid["nombre_normalizado"] == "UNIVERSIDAD ALBERTO HURTADO"), "rut_valido"].unique())

['73923400-k']
[ True]


Funcionó. Revisemos ahora cuántas filas faltan por imputar.

In [ ]:
df_anid[(~df_anid["rut_valido"]) & (df_anid["PAISINSTITUCION"] == "CHILE")]

,PROGRAMA,CONCURSO,FOLIO,AGNOPOSTULACION,AGNOFALLO,RESPONSABLEPROYECTO,SEXO,ROLINSTITUCION,INSTITUCION1ERNIVEL,RUTINSTITUCION,PAISINSTITUCION,REGIONINSTITUCION,SUBDIRECCION,rut_normalizado,rut_valido,nombre_normalizado
0,REGIONAL,II CONCURSO DE FORTALECIMIENTO A LA CONTINUIDA...,R15A10001,2015,2016,EDUARDO GRATACOS,HOMBRE,BENEFICIARIA PRINCIPAL,CENTRO REGIONAL DE INNOVACION HORTOFRUTICOLA ...,SIN INFORMACION,CHILE,REGION DE VALPARAISO,SCIA,-,False,CENTRO REGIONAL DE INNOVACION HORTOFRUTICOLA D...
1,REGIONAL,II CONCURSO DE FORTALECIMIENTO A LA CONTINUIDA...,R15A10002,2015,2016,LUIS ROJAS ARAYA,HOMBRE,BENEFICIARIA PRINCIPAL,CENTRO DE INVESTIGACION CIENTIFICO TECNOLOGICO...,SIN INFORMACION,CHILE,REGION DE ANTOFAGASTA,SCIA,-,False,CENTRO DE INVESTIGACION CIENTIFICO TECNOLOGICO...
3,REGIONAL,II CONCURSO DE FORTALECIMIENTO DE CENTROS REGI...,R16F10008,2016,2016,EDUARDO GRATACOS N.,HOMBRE,BENEFICIARIA PRINCIPAL,CENTRO REGIONAL DE INNOVACION HORTOFRUTICOLA ...,SIN INFORMACION,CHILE,REGION DE VALPARAISO,SCIA,-,False,CENTRO REGIONAL DE INNOVACION HORTOFRUTICOLA D...
4,REGIONAL,II CONCURSO DE FORTALECIMIENTO DE CENTROS REGI...,R16F10009,2016,2016,BERNARDO BROITMAN ROJAS,HOMBRE,BENEFICIARIA PRINCIPAL,CENTRO DE ESTUDIOS AVANZADOS EN ZONAS ARIDAS (...,SIN INFORMACION,CHILE,REGION DE COQUIMBO,SCIA,-,False,CENTRO DE ESTUDIOS AVANZADOS EN ZONAS ARIDAS (...
5,REGIONAL,II CONCURSO DE FORTALECIMIENTO DE CENTROS REGI...,R16F10002,2016,2016,CESAR BURGOS,HOMBRE,BENEFICIARIA PRINCIPAL,CENTRO DE GENOMICA NUTRICIONAL AGROACUICOLA (C...,SIN INFORMACION,CHILE,REGION DE LA ARAUCANIA,SCIA,-,False,CENTRO DE GENOMICA NUTRICIONAL AGROACUICOLA (C...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22954,PCI,FOMENTO A LA VINCULACION INTERNACIONAL PARA IN...,FOVI220124,2022,2022,MARISOL RUIZ CONTRERAS,MUJER,INSTITUCION ASOCIADA,ILUSTRE MUNICIPALIDAD DE VALDIVIA,-,CHILE,REGION DE LOS RIOS,SREC,-,False,ILUSTRE MUNICIPALIDAD DE VALDIVIA
22955,PCI,FOMENTO A LA VINCULACION INTERNACIONAL PARA IN...,FOVI220211,2022,2022,DANIEL MEDINA SALAS,HOMBRE,INSTITUCION ASOCIADA,SEREMI DE SALUD REGION DE LOS LAGOS,-,CHILE,NO REGISTRA REGION,SREC,-,False,SEREMI DE SALUD REGION DE LOS LAGOS
22956,PCI,FOMENTO A LA VINCULACION INTERNACIONAL PARA IN...,FOVI220211,2022,2022,DANIEL MEDINA SALAS,HOMBRE,INSTITUCION ASOCIADA,INSTITUTO NORUEGO DE INVESTIGACION DE AGUAS,-,CHILE,NO REGISTRA REGION,SREC,-,False,INSTITUTO NORUEGO DE INVESTIGACION DE AGUAS
23173,PCI,FOMENTO A LA VINCULACION INTERNACIONAL PARA IN...,FOVI220135,2022,2022,ARIEL LEIVA LOPEZ,HOMBRE,INSTITUCION EXTRANJERA,COMISION ECONOMICA PARA AMERICA LATINA Y EL CA...,NO APLICA,CHILE,REGION METROPOLITANA DE SANTIAGO,SREC,noaplica,False,COMISION ECONOMICA PARA AMERICA LATINA Y EL CA...


804 filas.

Veamos ahora cuántas instituciones chilenas tienen más de un RUT, después de los cambios que hicimos.

In [ ]:
for name in df_anid.loc[df_anid["PAISINSTITUCION"] == "CHILE", "nombre_normalizado"].unique():
  ruts = df_anid.loc[df_anid["nombre_normalizado"] == name, "rut_normalizado"].unique()
  if len(ruts) != 1:
    print(name, ruts, len(ruts))

UNIVERSIDAD DE LOS ANDES ['71614000-8' 'noaplica'] 2
UNIVERSIDAD DE LAS AMERICAS ['71540800-7' nan] 2
INDUPLAS LTDA ['76174097-0' '76534660-6'] 2
SERVICIO NACIONAL DE PESCA Y ACUICULTURA ['60701002-1' '-' '60701000-1'] 3
ANTOFAGASTA MINERALS SA ['93920000-2' '-'] 2
ESSBIO SA ['76833300-9' '96579330-5'] 2
CLINICA RED SALUD SANTIAGO ['96885930-7' '76020458-7'] 2
HOSPITAL DR ERNESTO TORRES GALDAMES ['62000530-4' '61606100-3'] 2
JICA CHILE ['noaplica' '69500700-0'] 2
INSTITUTO NACIONAL DE GERIATRIA ['61608403-8' 'noaplica'] 2
HOSPITAL CARLOS VAN BUREN ['61606500-9' '61602054-4'] 2
CENTRO DE ESTUDIOS DE CONFLICTO Y COHESION SOCIAL ['12265668-3' '-' '60910000-1'] 3
UNIVERSIDAD AUTONOMA DE BARCELONA ['818002-1' '-' 'noaplica'] 3
INSTITUT DE RECHERCHE POUR LE DEVELOPPEMENT ['72422800-3' 'noaplica' '-'] 3
EUROPEAN SOUTHERN OBSERVATORY ['-' nan] 2
UNIVERSIDAD DE LA REPUBLICA ['noaplica' '71528700-5' '-'] 3


Estos casos hay que analizarlos en mayor profundidad.

##### Revisión de casos raros

Veamos el caso de la Universidad de Los Andes

In [ ]:
df_anid[(df_anid["nombre_normalizado"] == "UNIVERSIDAD DE LOS ANDES") & (df_anid["rut_normalizado"] == "noaplica")]

,PROGRAMA,CONCURSO,FOLIO,AGNOPOSTULACION,AGNOFALLO,RESPONSABLEPROYECTO,SEXO,ROLINSTITUCION,INSTITUCION1ERNIVEL,RUTINSTITUCION,PAISINSTITUCION,REGIONINSTITUCION,SUBDIRECCION,rut_normalizado,rut_valido,nombre_normalizado
19101,PCI,PROYECTOS DE APOYO A LA FORMACION DE REDES INT...,REDES150042,2015,2016,XIMENA ZARATE BONILLA,MUJER,CENTRO EXTRANJERO ASOCIADO,UNIVERSIDAD DE LOS ANDES,NO APLICA,COLOMBIA,NO REGISTRA REGION,SREC,noaplica,False,UNIVERSIDAD DE LOS ANDES
19897,PCI,CONCURSO DE APOYO A LA FORMACION DE REDES INTE...,REDI170101,2017,2017,FERNANDO ROSENBLATT BOGACZ,HOMBRE,INSTITUCION EXTRANJERA,UNIVERSIDAD DE LOS ANDES,NO APLICA,COLOMBIA,NO REGISTRA REGION,SREC,noaplica,False,UNIVERSIDAD DE LOS ANDES


Es una coincidencia de nombre; es una universidad colombiana que lleva el mismo nombre que la institución chilena. Le cambiaremos el nombre para indicar que es una institución distinta.

In [ ]:
df_anid.loc[(df_anid["nombre_normalizado"] == "UNIVERSIDAD DE LOS ANDES") & (df_anid["rut_normalizado"] == "noaplica"), "nombre_normalizado"] = "UNIVERSIDAD DE LOS ANDES (COLOMBIA)"
df_anid.loc[df_anid["nombre_normalizado"] == "UNIVERSIDAD DE LOS ANDES (COLOMBIA)"]

,PROGRAMA,CONCURSO,FOLIO,AGNOPOSTULACION,AGNOFALLO,RESPONSABLEPROYECTO,SEXO,ROLINSTITUCION,INSTITUCION1ERNIVEL,RUTINSTITUCION,PAISINSTITUCION,REGIONINSTITUCION,SUBDIRECCION,rut_normalizado,rut_valido,nombre_normalizado
19101,PCI,PROYECTOS DE APOYO A LA FORMACION DE REDES INT...,REDES150042,2015,2016,XIMENA ZARATE BONILLA,MUJER,CENTRO EXTRANJERO ASOCIADO,UNIVERSIDAD DE LOS ANDES,NO APLICA,COLOMBIA,NO REGISTRA REGION,SREC,noaplica,False,UNIVERSIDAD DE LOS ANDES (COLOMBIA)
19897,PCI,CONCURSO DE APOYO A LA FORMACION DE REDES INTE...,REDI170101,2017,2017,FERNANDO ROSENBLATT BOGACZ,HOMBRE,INSTITUCION EXTRANJERA,UNIVERSIDAD DE LOS ANDES,NO APLICA,COLOMBIA,NO REGISTRA REGION,SREC,noaplica,False,UNIVERSIDAD DE LOS ANDES (COLOMBIA)


Veamos ahora el caso de la Universidad de Las Américas

In [ ]:
df_anid[(df_anid["nombre_normalizado"] == "UNIVERSIDAD DE LAS AMERICAS") & (df_anid["rut_normalizado"].isnull())]

,PROGRAMA,CONCURSO,FOLIO,AGNOPOSTULACION,AGNOFALLO,RESPONSABLEPROYECTO,SEXO,ROLINSTITUCION,INSTITUCION1ERNIVEL,RUTINSTITUCION,PAISINSTITUCION,REGIONINSTITUCION,SUBDIRECCION,rut_normalizado,rut_valido,nombre_normalizado
23635,PCI,PROGRAMAS REGIONALES STIC-AMSUD - MATH-AMSUD -...,AMSUD220010,2022,2023,SEBASTIAN REYES CAROCCA,MASCULINO,INSTITUCION SUDAMERICANA,UNIVERSIDAD DE LAS AMERICAS,NaN,ECUADOR,NaN,SREC,NaN,False,UNIVERSIDAD DE LAS AMERICAS


Es una universidad ecuatoriana, le cambiaremos el nombre.

In [ ]:
df_anid.loc[(df_anid["nombre_normalizado"] == "UNIVERSIDAD DE LAS AMERICAS") & (df_anid["rut_normalizado"].isnull()), "nombre_normalizado"] = "UNIVERSIDAD DE LAS AMERICAS (ECUADOR)"
df_anid.loc[df_anid["nombre_normalizado"] == "UNIVERSIDAD DE LAS AMERICAS (ECUADOR)"]

,PROGRAMA,CONCURSO,FOLIO,AGNOPOSTULACION,AGNOFALLO,RESPONSABLEPROYECTO,SEXO,ROLINSTITUCION,INSTITUCION1ERNIVEL,RUTINSTITUCION,PAISINSTITUCION,REGIONINSTITUCION,SUBDIRECCION,rut_normalizado,rut_valido,nombre_normalizado
23635,PCI,PROGRAMAS REGIONALES STIC-AMSUD - MATH-AMSUD -...,AMSUD220010,2022,2023,SEBASTIAN REYES CAROCCA,MASCULINO,INSTITUCION SUDAMERICANA,UNIVERSIDAD DE LAS AMERICAS,NaN,ECUADOR,NaN,SREC,NaN,False,UNIVERSIDAD DE LAS AMERICAS (ECUADOR)


Las siguientes instituciones tienen más de un RUT válido asociado, así que buscaremos manualmente cuál es su RUT real:

- INDUPLAS LTDA
- SERVICIO NACIONAL DE PESCA Y ACUICULTURA
- ESSBIO SA
- CLINICA RED SALUD SANTIAGO
- HOSPITAL DR ERNESTO TORRES GALDAMES
- HOSPITAL CARLOS VAN BUREN
- CENTRO DE ESTUDIOS DE CONFLICTO Y COHESION SOCIAL



In [ ]:
names_to_fix = [("INDUPLAS LTDA", "76174097-0"),
                ("SERVICIO NACIONAL DE PESCA Y ACUICULTURA", "60701002-1"),
                ("ESSBIO SA", "76833300-9"),
                ("CLINICA RED SALUD SANTIAGO", "76020458-7"),
                ("HOSPITAL DR ERNESTO TORRES GALDAMES", "62000530-4"),
                ("HOSPITAL CARLOS VAN BUREN", "61602054-4"),
                ("CENTRO DE ESTUDIOS DE CONFLICTO Y COHESION SOCIAL", "60910000-1")]

for (name, rut) in names_to_fix:
  df_anid.loc[df_anid["nombre_normalizado"] == name, "rut_normalizado"] = rut

Revisaremos que haya funcionado

In [ ]:
df_anid.loc[df_anid["nombre_normalizado"] == "HOSPITAL CARLOS VAN BUREN"]

,PROGRAMA,CONCURSO,FOLIO,AGNOPOSTULACION,AGNOFALLO,RESPONSABLEPROYECTO,SEXO,ROLINSTITUCION,INSTITUCION1ERNIVEL,RUTINSTITUCION,PAISINSTITUCION,REGIONINSTITUCION,SUBDIRECCION,rut_normalizado,rut_valido,nombre_normalizado
1684,FONIS,XIII CONCURSO PROYECTOS INVESTIGACION Y DESARR...,SA16I0003,2016,2016,PABLO MOYA VERA,HOMBRE,INSTITUCION ASOCIADA,HOSPITAL CARLOS VAN BUREN,61606500-9,NO REGISTRA PAIS,NO REGISTRA REGION,SIA,61602054-4,True,HOSPITAL CARLOS VAN BUREN
2228,FONIS,XIV CONCURSO NACIONAL DE PROYECTOS DE INVESTIG...,SA17I0124,2017,2017,CARLOS BENNETT COLOMER,HOMBRE,ENTIDAD ASOCIADA,HOSPITAL CARLOS VAN BUREN,61606500-9,NO REGISTRA PAIS,NO REGISTRA REGION,SIA,61602054-4,True,HOSPITAL CARLOS VAN BUREN
3318,FONIS,XVI CONCURSO NACIONAL DE PROYECTOS DE INVESTIG...,SA19I0191,2019,2019,RODRIGO VIDAL SOBARZO,HOMBRE,ENTIDAD ASOCIADA,HOSPITAL CARLOS VAN BUREN,61606500-9,NO REGISTRA PAIS,NO REGISTRA REGION,SIA,61602054-4,True,HOSPITAL CARLOS VAN BUREN
3722,FONIS,XVII CONCURSO NACIONAL DE PROYECTOS DE INVESTI...,SA20I0095,2020,2020,ALONSO ORTEGA GONZALEZ,HOMBRE,ENTIDAD ASOCIADA,HOSPITAL CARLOS VAN BUREN,61606500-9,NO REGISTRA PAIS,NO REGISTRA REGION,SIA,61602054-4,True,HOSPITAL CARLOS VAN BUREN
4689,FONIS,XIX CONCURSO NACIONAL DE PROYECTOS DE INVESTIG...,SA22I0035,2022,2022,MARIA ELENA CEBALLOS VALDIVIELSO,MUJER,ENTIDAD ASOCIADA,HOSPITAL CARLOS VAN BUREN,61602054-4,CHILE,NO REGISTRA REGION,SIA,61602054-4,True,HOSPITAL CARLOS VAN BUREN
5810,FONDEF,CONCURSO IDEA I+D 2023,ID23I10327,2022,2023,CARLA TARAMASCO TORO,MUJER,ENTIDAD ASOCIADA,HOSPITAL CARLOS VAN BUREN,61602054-4,CHILE,REGION DE VALPARAISO,SIA,61602054-4,True,HOSPITAL CARLOS VAN BUREN


Todas las filas tienen el mismo rut, funcionó. Revisemos ahora Antofagasta Minerals.

In [ ]:
df_anid.loc[df_anid["nombre_normalizado"] == "ANTOFAGASTA MINERALS SA"]

,PROGRAMA,CONCURSO,FOLIO,AGNOPOSTULACION,AGNOFALLO,RESPONSABLEPROYECTO,SEXO,ROLINSTITUCION,INSTITUCION1ERNIVEL,RUTINSTITUCION,PAISINSTITUCION,REGIONINSTITUCION,SUBDIRECCION,rut_normalizado,rut_valido,nombre_normalizado
2458,FONDEF,CONCURSO IDEA I+D 2018,ID18I10117,2018,2018,LEOPOLDO GUTIERREZ BRIONES,HOMBRE,ENTIDAD ASOCIADA,ANTOFAGASTA MINERALS S.A.,93920000-2,CHILE,REGION METROPOLITANA DE SANTIAGO,SIA,93920000-2,True,ANTOFAGASTA MINERALS SA
4012,FONDEF,CONCURSO IDEA I+D 2021,ID21I10111,2021,2021,IVAN NANCUCHEO CUEVAS,HOMBRE,ENTIDAD ASOCIADA,ANTOFAGASTA MINERALS S.A.,93920000-2,CHILE,REGION METROPOLITANA DE SANTIAGO,SIA,93920000-2,True,ANTOFAGASTA MINERALS SA
5033,FONDEF,CONCURSO IDEA I+D 2022,ID22I10190,2022,2022,LILIAN VELASQUEZ YEVENES,MUJER,COLABORADORA,ANTOFAGASTA MINERALS S.A.,93920000-2,CHILE,REGION METROPOLITANA DE SANTIAGO,SIA,93920000-2,True,ANTOFAGASTA MINERALS SA
6233,FONDECYT,FONDECYT REGULAR 2016,1160547,2015,2016,JUAN YIANATOS BERNARDINO,HOMBRE,INSTITUCION APORTANTE,ANTOFAGASTA MINERALS S.A.,-,NO REGISTRA PAIS,NO REGISTRA REGION,SPI,-,False,ANTOFAGASTA MINERALS SA


La fila sin RUT es porque no registra país. Arreglaremos eso.

In [ ]:
df_anid.loc[(df_anid["nombre_normalizado"] == "ANTOFAGASTA MINERALS SA") & (df_anid["rut_normalizado"] == "-"),
            ["PAISINSTITUCION", "rut_normalizado", "rut_valido"]] = ("CHILE", "93920000-2", True)

df_anid.loc[df_anid["nombre_normalizado"] == "ANTOFAGASTA MINERALS SA"]

,PROGRAMA,CONCURSO,FOLIO,AGNOPOSTULACION,AGNOFALLO,RESPONSABLEPROYECTO,SEXO,ROLINSTITUCION,INSTITUCION1ERNIVEL,RUTINSTITUCION,PAISINSTITUCION,REGIONINSTITUCION,SUBDIRECCION,rut_normalizado,rut_valido,nombre_normalizado
2458,FONDEF,CONCURSO IDEA I+D 2018,ID18I10117,2018,2018,LEOPOLDO GUTIERREZ BRIONES,HOMBRE,ENTIDAD ASOCIADA,ANTOFAGASTA MINERALS S.A.,93920000-2,CHILE,REGION METROPOLITANA DE SANTIAGO,SIA,93920000-2,True,ANTOFAGASTA MINERALS SA
4012,FONDEF,CONCURSO IDEA I+D 2021,ID21I10111,2021,2021,IVAN NANCUCHEO CUEVAS,HOMBRE,ENTIDAD ASOCIADA,ANTOFAGASTA MINERALS S.A.,93920000-2,CHILE,REGION METROPOLITANA DE SANTIAGO,SIA,93920000-2,True,ANTOFAGASTA MINERALS SA
5033,FONDEF,CONCURSO IDEA I+D 2022,ID22I10190,2022,2022,LILIAN VELASQUEZ YEVENES,MUJER,COLABORADORA,ANTOFAGASTA MINERALS S.A.,93920000-2,CHILE,REGION METROPOLITANA DE SANTIAGO,SIA,93920000-2,True,ANTOFAGASTA MINERALS SA
6233,FONDECYT,FONDECYT REGULAR 2016,1160547,2015,2016,JUAN YIANATOS BERNARDINO,HOMBRE,INSTITUCION APORTANTE,ANTOFAGASTA MINERALS S.A.,-,CHILE,NO REGISTRA REGION,SPI,93920000-2,True,ANTOFAGASTA MINERALS SA


Veamos ahora el caso de Instituto Nacional de Geriatría.

In [ ]:
df_anid.loc[df_anid["nombre_normalizado"] == "INSTITUTO NACIONAL DE GERIATRIA"]

,PROGRAMA,CONCURSO,FOLIO,AGNOPOSTULACION,AGNOFALLO,RESPONSABLEPROYECTO,SEXO,ROLINSTITUCION,INSTITUCION1ERNIVEL,RUTINSTITUCION,PAISINSTITUCION,REGIONINSTITUCION,SUBDIRECCION,rut_normalizado,rut_valido,nombre_normalizado
3596,FONDEF,CONCURSO IDEA I+D 2020,ID20I10234,2020,2020,ANDREA PAULA-LIMA -,MUJER,ENTIDAD ASOCIADA,INSTITUTO NACIONAL DE GERIATRIA,61608403-8,CHILE,NO REGISTRA REGION,SIA,61608403-8,True,INSTITUTO NACIONAL DE GERIATRIA
19466,PCI,APOYO A LA FORMACION DE REDES INTERNACIONALES ...,REDES170002,2017,2017,IVAN PALOMO GONZALEZ,HOMBRE,CENTRO EXTRANJERO ASOCIADO,INSTITUTO NACIONAL DE GERIATRIA,NO APLICA,MEXICO,NO REGISTRA REGION,SREC,noaplica,False,INSTITUTO NACIONAL DE GERIATRIA
19471,PCI,APOYO A LA FORMACION DE REDES INTERNACIONALES ...,REDES170003,2017,2017,EDUARDO FUENTES QUINTEROS,HOMBRE,CENTRO EXTRANJERO ASOCIADO,INSTITUTO NACIONAL DE GERIATRIA,NO APLICA,MEXICO,NO REGISTRA REGION,SREC,noaplica,False,INSTITUTO NACIONAL DE GERIATRIA


Nuevamente, coincidencia de nombres: es un centro de México. Arreglaremos eso.

In [ ]:
df_anid.loc[(df_anid["nombre_normalizado"] == "INSTITUTO NACIONAL DE GERIATRIA") & (~df_anid["rut_valido"]), "nombre_normalizado"] = "INSTITUTO NACIONAL DE GERIATRIA (MEXICO)"
df_anid.loc[df_anid["nombre_normalizado"].str.startswith("INSTITUTO NACIONAL DE GERIATRIA")]

,PROGRAMA,CONCURSO,FOLIO,AGNOPOSTULACION,AGNOFALLO,RESPONSABLEPROYECTO,SEXO,ROLINSTITUCION,INSTITUCION1ERNIVEL,RUTINSTITUCION,PAISINSTITUCION,REGIONINSTITUCION,SUBDIRECCION,rut_normalizado,rut_valido,nombre_normalizado
3596,FONDEF,CONCURSO IDEA I+D 2020,ID20I10234,2020,2020,ANDREA PAULA-LIMA -,MUJER,ENTIDAD ASOCIADA,INSTITUTO NACIONAL DE GERIATRIA,61608403-8,CHILE,NO REGISTRA REGION,SIA,61608403-8,True,INSTITUTO NACIONAL DE GERIATRIA
19466,PCI,APOYO A LA FORMACION DE REDES INTERNACIONALES ...,REDES170002,2017,2017,IVAN PALOMO GONZALEZ,HOMBRE,CENTRO EXTRANJERO ASOCIADO,INSTITUTO NACIONAL DE GERIATRIA,NO APLICA,MEXICO,NO REGISTRA REGION,SREC,noaplica,False,INSTITUTO NACIONAL DE GERIATRIA (MEXICO)
19471,PCI,APOYO A LA FORMACION DE REDES INTERNACIONALES ...,REDES170003,2017,2017,EDUARDO FUENTES QUINTEROS,HOMBRE,CENTRO EXTRANJERO ASOCIADO,INSTITUTO NACIONAL DE GERIATRIA,NO APLICA,MEXICO,NO REGISTRA REGION,SREC,noaplica,False,INSTITUTO NACIONAL DE GERIATRIA (MEXICO)


Veamos ahora la Universidad de La República.

In [ ]:
df_anid.loc[df_anid["nombre_normalizado"] == "UNIVERSIDAD DE LA REPUBLICA"]

,PROGRAMA,CONCURSO,FOLIO,AGNOPOSTULACION,AGNOFALLO,RESPONSABLEPROYECTO,SEXO,ROLINSTITUCION,INSTITUCION1ERNIVEL,RUTINSTITUCION,PAISINSTITUCION,REGIONINSTITUCION,SUBDIRECCION,rut_normalizado,rut_valido,nombre_normalizado
19250,PCI,PROYECTOS DE APOYO A LA FORMACION DE REDES INT...,REDES150150,2015,2016,JOSE EGANA TOMIC,HOMBRE,CENTRO EXTRANJERO PRINCIPAL,UNIVERSIDAD DE LA REPUBLICA,NO APLICA,URUGUAY,NO REGISTRA REGION,SREC,noaplica,False,UNIVERSIDAD DE LA REPUBLICA
19472,PCI,APOYO A LA FORMACION DE REDES INTERNACIONALES ...,REDES170003,2017,2017,EDUARDO FUENTES QUINTEROS,HOMBRE,CENTRO EXTRANJERO ASOCIADO,UNIVERSIDAD DE LA REPUBLICA,NO APLICA,URUGUAY,NO REGISTRA REGION,SREC,noaplica,False,UNIVERSIDAD DE LA REPUBLICA
20019,PCI,CONCURSO DE APOYO A LA FORMACION DE REDES INTE...,REDI170225,2017,2017,PABLO GUINDOS BRETONES,HOMBRE,INSTITUCION EXTRANJERA,UNIVERSIDAD DE LA REPUBLICA,NO APLICA,URUGUAY,NO REGISTRA REGION,SREC,noaplica,False,UNIVERSIDAD DE LA REPUBLICA
20200,PCI,CONCURSO DE APOYO A LA FORMACION DE REDES INTE...,REDI170444,2017,2017,PEDRO OLIVARES SANCHEZ TOLEDO,HOMBRE,INSTITUCION EXTRANJERA,UNIVERSIDAD DE LA REPUBLICA,NO APLICA,URUGUAY,NO REGISTRA REGION,SREC,noaplica,False,UNIVERSIDAD DE LA REPUBLICA
20618,PCI,APOYO A LA FORMACION DE REDES INTERNACIONALES ...,REDES180024,2018,2018,JOSE EGANA TOMIC,HOMBRE,CENTRO EXTRANJERO ASOCIADO,UNIVERSIDAD DE LA REPUBLICA,NO APLICA,URUGUAY,NO REGISTRA REGION,SREC,noaplica,False,UNIVERSIDAD DE LA REPUBLICA
20924,PCI,ATRACCION DE CAPITAL HUMANO AVANZADO DEL EXTRA...,MEC80180040,2018,2018,LEONARDO BARDEHLE PARRA,HOMBRE,INSTITUCION DE PROCEDENCIA,UNIVERSIDAD DE LA REPUBLICA,NO APLICA,URUGUAY,NO REGISTRA REGION,SREC,noaplica,False,UNIVERSIDAD DE LA REPUBLICA
21802,REC,CONCURSO PARA LA ASIGNACION RAPIDA DE RECURSOS...,COVID0343,2020,2020,CLAUDIA CALQUIN DONOSO,MUJER,ENTIDAD ASOCIADA,UNIVERSIDAD DE LA REPUBLICA,71528700-5,CHILE,REGION METROPOLITANA DE SANTIAGO,SREC,71528700-5,True,UNIVERSIDAD DE LA REPUBLICA
22110,PCI,CONCURSO DEL PROGRAMA REGIONAL CLIMAT/AMSUD - ...,CLI2020004,2020,2021,GERMAN REIG CARDARELLA,HOMBRE,INSTITUCION SUDAMERICANA,UNIVERSIDAD DE LA REPUBLICA,-,URUGUAY,NO REGISTRA REGION,SREC,-,False,UNIVERSIDAD DE LA REPUBLICA
22150,PCI,CONCURSO DEL PROGRAMA REGIONAL CLIMAT/AMSUD - ...,CLI2020015,2020,2021,ALEJANDRO MAASS SEPULVEDA,HOMBRE,INSTITUCION SUDAMERICANA,UNIVERSIDAD DE LA REPUBLICA,-,URUGUAY,NO REGISTRA REGION,SREC,-,False,UNIVERSIDAD DE LA REPUBLICA
22191,PCI,CONCURSO DEL PROGRAMA REGIONAL MATH/AMSUD - AN...,MATH2020018,2020,2021,DANIEL BARRERA SALAZAR,HOMBRE,INSTITUCION SUDAMERICANA,UNIVERSIDAD DE LA REPUBLICA,-,URUGUAY,NO REGISTRA REGION,SREC,-,False,UNIVERSIDAD DE LA REPUBLICA


Nuevamente, coincidencia de nombre. De hecho, la institución chilena se llama Universidad La República, no "de La República". Arreglaremos eso.

In [ ]:
df_anid.loc[(df_anid["nombre_normalizado"] == "UNIVERSIDAD DE LA REPUBLICA") & (df_anid["rut_valido"]), "nombre_normalizado"] = "UNIVERSIDAD LA REPUBLICA"
df_anid.loc[df_anid["nombre_normalizado"].isin(("UNIVERSIDAD DE LA REPUBLICA", "UNIVERSIDAD LA REPUBLICA"))]

,PROGRAMA,CONCURSO,FOLIO,AGNOPOSTULACION,AGNOFALLO,RESPONSABLEPROYECTO,SEXO,ROLINSTITUCION,INSTITUCION1ERNIVEL,RUTINSTITUCION,PAISINSTITUCION,REGIONINSTITUCION,SUBDIRECCION,rut_normalizado,rut_valido,nombre_normalizado
19250,PCI,PROYECTOS DE APOYO A LA FORMACION DE REDES INT...,REDES150150,2015,2016,JOSE EGANA TOMIC,HOMBRE,CENTRO EXTRANJERO PRINCIPAL,UNIVERSIDAD DE LA REPUBLICA,NO APLICA,URUGUAY,NO REGISTRA REGION,SREC,noaplica,False,UNIVERSIDAD DE LA REPUBLICA
19472,PCI,APOYO A LA FORMACION DE REDES INTERNACIONALES ...,REDES170003,2017,2017,EDUARDO FUENTES QUINTEROS,HOMBRE,CENTRO EXTRANJERO ASOCIADO,UNIVERSIDAD DE LA REPUBLICA,NO APLICA,URUGUAY,NO REGISTRA REGION,SREC,noaplica,False,UNIVERSIDAD DE LA REPUBLICA
20019,PCI,CONCURSO DE APOYO A LA FORMACION DE REDES INTE...,REDI170225,2017,2017,PABLO GUINDOS BRETONES,HOMBRE,INSTITUCION EXTRANJERA,UNIVERSIDAD DE LA REPUBLICA,NO APLICA,URUGUAY,NO REGISTRA REGION,SREC,noaplica,False,UNIVERSIDAD DE LA REPUBLICA
20200,PCI,CONCURSO DE APOYO A LA FORMACION DE REDES INTE...,REDI170444,2017,2017,PEDRO OLIVARES SANCHEZ TOLEDO,HOMBRE,INSTITUCION EXTRANJERA,UNIVERSIDAD DE LA REPUBLICA,NO APLICA,URUGUAY,NO REGISTRA REGION,SREC,noaplica,False,UNIVERSIDAD DE LA REPUBLICA
20618,PCI,APOYO A LA FORMACION DE REDES INTERNACIONALES ...,REDES180024,2018,2018,JOSE EGANA TOMIC,HOMBRE,CENTRO EXTRANJERO ASOCIADO,UNIVERSIDAD DE LA REPUBLICA,NO APLICA,URUGUAY,NO REGISTRA REGION,SREC,noaplica,False,UNIVERSIDAD DE LA REPUBLICA
20924,PCI,ATRACCION DE CAPITAL HUMANO AVANZADO DEL EXTRA...,MEC80180040,2018,2018,LEONARDO BARDEHLE PARRA,HOMBRE,INSTITUCION DE PROCEDENCIA,UNIVERSIDAD DE LA REPUBLICA,NO APLICA,URUGUAY,NO REGISTRA REGION,SREC,noaplica,False,UNIVERSIDAD DE LA REPUBLICA
21802,REC,CONCURSO PARA LA ASIGNACION RAPIDA DE RECURSOS...,COVID0343,2020,2020,CLAUDIA CALQUIN DONOSO,MUJER,ENTIDAD ASOCIADA,UNIVERSIDAD DE LA REPUBLICA,71528700-5,CHILE,REGION METROPOLITANA DE SANTIAGO,SREC,71528700-5,True,UNIVERSIDAD LA REPUBLICA
22110,PCI,CONCURSO DEL PROGRAMA REGIONAL CLIMAT/AMSUD - ...,CLI2020004,2020,2021,GERMAN REIG CARDARELLA,HOMBRE,INSTITUCION SUDAMERICANA,UNIVERSIDAD DE LA REPUBLICA,-,URUGUAY,NO REGISTRA REGION,SREC,-,False,UNIVERSIDAD DE LA REPUBLICA
22150,PCI,CONCURSO DEL PROGRAMA REGIONAL CLIMAT/AMSUD - ...,CLI2020015,2020,2021,ALEJANDRO MAASS SEPULVEDA,HOMBRE,INSTITUCION SUDAMERICANA,UNIVERSIDAD DE LA REPUBLICA,-,URUGUAY,NO REGISTRA REGION,SREC,-,False,UNIVERSIDAD DE LA REPUBLICA
22191,PCI,CONCURSO DEL PROGRAMA REGIONAL MATH/AMSUD - AN...,MATH2020018,2020,2021,DANIEL BARRERA SALAZAR,HOMBRE,INSTITUCION SUDAMERICANA,UNIVERSIDAD DE LA REPUBLICA,-,URUGUAY,NO REGISTRA REGION,SREC,-,False,UNIVERSIDAD DE LA REPUBLICA


Veremos ahora el caso de Jica Chile.

In [ ]:
df_anid[df_anid["nombre_normalizado"] == "JICA CHILE"]

,PROGRAMA,CONCURSO,FOLIO,AGNOPOSTULACION,AGNOFALLO,RESPONSABLEPROYECTO,SEXO,ROLINSTITUCION,INSTITUCION1ERNIVEL,RUTINSTITUCION,PAISINSTITUCION,REGIONINSTITUCION,SUBDIRECCION,rut_normalizado,rut_valido,nombre_normalizado
2143,FONDEF,PRIMER CONCURSO INVESTIGACION TECNOLOGICA TEM...,IT17F10001,2017,2017,CARLOS RIQUELME SALAMANCA,HOMBRE,ENTIDAD ASOCIADA,JICA CHILE,NO APLICA,JAPON,NO REGISTRA REGION,SIA,noaplica,False,JICA CHILE
3486,FONDEF,CONCURSO IDEA I+D 2020,ID20I10085,2020,2020,CARLOS RIQUELME SALAMANCA,HOMBRE,ENTIDAD ASOCIADA,JICA CHILE,69500700-0,CHILE,NO REGISTRA REGION,SIA,69500700-0,True,JICA CHILE


JICA Chile es la Agencia de Cooperacion Internacional del Japon; sin embargo, dado que tiene RUT, la consideraremos como entidad chilena.

In [ ]:
df_anid.loc[df_anid["nombre_normalizado"] == "JICA CHILE",
            ["PAISINSTITUCION", "rut_normalizado", "rut_valido"]] = ("CHILE", "69500700-0", True)

df_anid.loc[df_anid["nombre_normalizado"] == "JICA CHILE"]

,PROGRAMA,CONCURSO,FOLIO,AGNOPOSTULACION,AGNOFALLO,RESPONSABLEPROYECTO,SEXO,ROLINSTITUCION,INSTITUCION1ERNIVEL,RUTINSTITUCION,PAISINSTITUCION,REGIONINSTITUCION,SUBDIRECCION,rut_normalizado,rut_valido,nombre_normalizado
2143,FONDEF,PRIMER CONCURSO INVESTIGACION TECNOLOGICA TEM...,IT17F10001,2017,2017,CARLOS RIQUELME SALAMANCA,HOMBRE,ENTIDAD ASOCIADA,JICA CHILE,NO APLICA,CHILE,NO REGISTRA REGION,SIA,69500700-0,True,JICA CHILE
3486,FONDEF,CONCURSO IDEA I+D 2020,ID20I10085,2020,2020,CARLOS RIQUELME SALAMANCA,HOMBRE,ENTIDAD ASOCIADA,JICA CHILE,69500700-0,CHILE,NO REGISTRA REGION,SIA,69500700-0,True,JICA CHILE


Univesidad Autónoma de Barcelona: debe ser siempre de España, con rut "noaplica"

In [ ]:
df_anid.loc[df_anid["nombre_normalizado"] == "UNIVERSIDAD AUTONOMA DE BARCELONA",
            ["PAISINSTITUCION", "rut_normalizado", "rut_valido"]] = ("ESPANA", "noaplica", False)

df_anid.loc[df_anid["nombre_normalizado"] == "UNIVERSIDAD AUTONOMA DE BARCELONA"]

,PROGRAMA,CONCURSO,FOLIO,AGNOPOSTULACION,AGNOFALLO,RESPONSABLEPROYECTO,SEXO,ROLINSTITUCION,INSTITUCION1ERNIVEL,RUTINSTITUCION,PAISINSTITUCION,REGIONINSTITUCION,SUBDIRECCION,rut_normalizado,rut_valido,nombre_normalizado
4045,FONDEF,CONCURSO IDEA I+D 2021,ID21I10130,2021,2021,ERICO CARMONA ORTIZ,HOMBRE,ENTIDAD ASOCIADA,UNIVERSIDAD AUTONOMA DE BARCELONA,818002-1,ESPANA,NO APLICA,SIA,noaplica,False,UNIVERSIDAD AUTONOMA DE BARCELONA
12842,FONDECYT,FONDECYT DE INICIACION EN INVESTIGACION 2020,11200398,2020,2020,DIOGENES HERNANDEZ ESPINOZA,HOMBRE,INSTITUCION APORTANTE,UNIVERSIDAD AUTONOMA DE BARCELONA,-,ESPANA,NO REGISTRA REGION,SPI,noaplica,False,UNIVERSIDAD AUTONOMA DE BARCELONA
17126,FONDECYT,FONDECYT INICIACION 2023,11230475,2022,2023,KATTIA NUÑEZ MONTERO,MUJER,INSTITUCION APORTANTE,UNIVERSIDAD AUTONOMA DE BARCELONA,818002-1,ESPANA,NaN,SPI,noaplica,False,UNIVERSIDAD AUTONOMA DE BARCELONA
18955,PAI,ATRACCION DE CAPITAL HUMANO AVANZADO DEL EXTRA...,PAI80160077,2016,2016,JOEL SAAVEDRA ALVEAR,HOMBRE,INSTITUCION EXTRANJERA ASOCIADA,UNIVERSIDAD AUTONOMA DE BARCELONA,NO APLICA,ESPANA,NO REGISTRA REGION,SREC,noaplica,False,UNIVERSIDAD AUTONOMA DE BARCELONA
19691,PCI,ATRACCION DE CAPITAL HUMANO AVANZADO DEL EXTRA...,MEC80170022,2017,2017,CARLOS MANTEROLA DELGADO,HOMBRE,INSTITUCION DE PROCEDENCIA,UNIVERSIDAD AUTONOMA DE BARCELONA,NO APLICA,ESPANA,NO REGISTRA REGION,SREC,noaplica,False,UNIVERSIDAD AUTONOMA DE BARCELONA
19927,PCI,CONCURSO DE APOYO A LA FORMACION DE REDES INTE...,REDI170133,2017,2017,IRENE SALVO AGOGLIA,MUJER,INSTITUCION EXTRANJERA,UNIVERSIDAD AUTONOMA DE BARCELONA,NO APLICA,ESPANA,NO REGISTRA REGION,SREC,noaplica,False,UNIVERSIDAD AUTONOMA DE BARCELONA
20969,PCI,ATRACCION DE CAPITAL HUMANO AVANZADO DEL EXTRA...,MEC80180078,2018,2018,VICENTE SISTO,HOMBRE,INSTITUCION DE PROCEDENCIA,UNIVERSIDAD AUTONOMA DE BARCELONA,NO APLICA,ESPANA,NO REGISTRA REGION,SREC,noaplica,False,UNIVERSIDAD AUTONOMA DE BARCELONA
21379,PCI,APOYO A LA FORMACION DE REDES INTERNACIONALES ...,REDES190135,2019,2019,EDMUNDO MUNOZ ALVEAR,HOMBRE,CENTRO EXTRANJERO PRINCIPAL,UNIVERSIDAD AUTONOMA DE BARCELONA,NO APLICA,ESPANA,NO REGISTRA REGION,SREC,noaplica,False,UNIVERSIDAD AUTONOMA DE BARCELONA
21502,PCI,ATRACCION DE CAPITAL HUMANO AVANZADO DEL EXTRA...,MEC80190026,2019,2019,CARLOS MANTEROLA DELGADO,HOMBRE,INSTITUCION DE PROCEDENCIA,UNIVERSIDAD AUTONOMA DE BARCELONA,NO APLICA,ESPANA,NO REGISTRA REGION,SREC,noaplica,False,UNIVERSIDAD AUTONOMA DE BARCELONA
21566,PCI,ATRACCION DE CAPITAL HUMANO AVANZADO DEL EXTRA...,MEC80190099,2019,2019,SEBASTIAN IBARRA GONZALEZ,HOMBRE,INSTITUCION DE PROCEDENCIA,UNIVERSIDAD AUTONOMA DE BARCELONA,NO APLICA,ESPANA,NO REGISTRA REGION,SREC,noaplica,False,UNIVERSIDAD AUTONOMA DE BARCELONA


INSTITUT DE RECHERCHE POUR LE DEVELOPPEMENT: debe ser siempre de Francia.

In [ ]:
df_anid.loc[df_anid["nombre_normalizado"] == "INSTITUT DE RECHERCHE POUR LE DEVELOPPEMENT",
            ["PAISINSTITUCION", "rut_normalizado", "rut_valido"]] = ("FRANCIA", "noaplica", False)
df_anid.loc[df_anid["nombre_normalizado"] == "INSTITUT DE RECHERCHE POUR LE DEVELOPPEMENT"]

,PROGRAMA,CONCURSO,FOLIO,AGNOPOSTULACION,AGNOFALLO,RESPONSABLEPROYECTO,SEXO,ROLINSTITUCION,INSTITUCION1ERNIVEL,RUTINSTITUCION,PAISINSTITUCION,REGIONINSTITUCION,SUBDIRECCION,rut_normalizado,rut_valido,nombre_normalizado
18918,PAI,ATRACCION DE CAPITAL HUMANO AVANZADO DEL EXTRA...,PAI80160043,2016,2016,LIZBETH NUNEZ CARRASCO,MUJER,INSTITUCION EXTRANJERA ASOCIADA,INSTITUT DE RECHERCHE POUR LE DEVELOPPEMENT,72422800-3,FRANCIA,NO REGISTRA REGION,SREC,noaplica,False,INSTITUT DE RECHERCHE POUR LE DEVELOPPEMENT
19635,PCI,APOYO A LA FORMACION DE REDES INTERNACIONALES ...,REDES170174,2017,2017,FERNANDO MARTINEZ ORTIZ,HOMBRE,CENTRO EXTRANJERO ASOCIADO,INSTITUT DE RECHERCHE POUR LE DEVELOPPEMENT,NO APLICA,FRANCIA,NO REGISTRA REGION,SREC,noaplica,False,INSTITUT DE RECHERCHE POUR LE DEVELOPPEMENT
22203,PCI,CONCURSO DEL PROGRAMA REGIONAL STIC/AMSUD - AN...,STIC2020008,2020,2021,VINICIUS RAMOS HENRIQUES MARACAJA COUTINHO,HOMBRE,INSTITUCION DE FRANCIA,INSTITUT DE RECHERCHE POUR LE DEVELOPPEMENT,-,FRANCIA,NO REGISTRA REGION,SREC,noaplica,False,INSTITUT DE RECHERCHE POUR LE DEVELOPPEMENT
23320,REC,PROGRAMAS REGIONALES STIC-AMSUD - MATH-AMSUD ...,AMSUD210025,2021,2022,FRANÇOIS CARLOTTI,HOMBRE,INSTITUCION DE FRANCIA,INSTITUT DE RECHERCHE POUR LE DEVELOPPEMENT,-,FRANCIA,NO APLICA,SREC,noaplica,False,INSTITUT DE RECHERCHE POUR LE DEVELOPPEMENT
23628,PCI,PROGRAMAS REGIONALES STIC-AMSUD - MATH-AMSUD -...,AMSUD220008,2022,2023,CLEMENTINE JUNQUAS,FEMENINO,INSTITUCION DE FRANCIA,INSTITUT DE RECHERCHE POUR LE DEVELOPPEMENT,-,FRANCIA,NaN,SREC,noaplica,False,INSTITUT DE RECHERCHE POUR LE DEVELOPPEMENT


EUROPEAN SOUTHERN OBSERVATORY: debe ser siempre Alemania.

In [ ]:
df_anid.loc[df_anid["nombre_normalizado"] == "EUROPEAN SOUTHERN OBSERVATORY",
            ["PAISINSTITUCION", "rut_normalizado", "rut_valido"]] = ("ALEMANIA", "noaplica", False)
df_anid.loc[df_anid["nombre_normalizado"] == "EUROPEAN SOUTHERN OBSERVATORY"]

,PROGRAMA,CONCURSO,FOLIO,AGNOPOSTULACION,AGNOFALLO,RESPONSABLEPROYECTO,SEXO,ROLINSTITUCION,INSTITUCION1ERNIVEL,RUTINSTITUCION,PAISINSTITUCION,REGIONINSTITUCION,SUBDIRECCION,rut_normalizado,rut_valido,nombre_normalizado
4151,FONDEF,CONCURSO IDEA I+D 2021,ID21I10228,2021,2021,PATRICIO GALEAS ECHEVERRIA,HOMBRE,ENTIDAD ASOCIADA,EUROPEAN SOUTHERN OBSERVATORY,-,ALEMANIA,NO APLICA,SIA,noaplica,False,EUROPEAN SOUTHERN OBSERVATORY
6088,FONDECYT,FONDECYT REGULAR 2016,1160236,2015,2016,ANDRES GUESALAGA MEISSNER,HOMBRE,INSTITUCION APORTANTE,EUROPEAN SOUTHERN OBSERVATORY,NaN,ALEMANIA,NO REGISTRA REGION,SPI,noaplica,False,EUROPEAN SOUTHERN OBSERVATORY
10224,FONDECYT,FONDECYT REGULAR 2019,1190186,2018,2019,ANDRES GUESALAGA MEISSNER,HOMBRE,INSTITUCION APORTANTE,EUROPEAN SOUTHERN OBSERVATORY,-,ALEMANIA,NO APLICA,SPI,noaplica,False,EUROPEAN SOUTHERN OBSERVATORY
14102,FONDECYT,FONDECYT REGULAR 2021,1211162,2020,2021,LEONARDO VANZI,HOMBRE,INSTITUCION APORTANTE,EUROPEAN SOUTHERN OBSERVATORY,-,ALEMANIA,NO APLICA,SPI,noaplica,False,EUROPEAN SOUTHERN OBSERVATORY
21085,ASTRONOMIA,FONDO ASTRONOMIA QUIMAL 2019,QUIMAL190009,2019,2019,PATRICIO GALEAS ECHEVERRIA,HOMBRE,INSTITUCION APORTANTE,EUROPEAN SOUTHERN OBSERVATORY,SIN INFORMACION,ALEMANIA,REGION METROPOLITANA DE SANTIAGO,SREC,noaplica,False,EUROPEAN SOUTHERN OBSERVATORY
22075,PCI,POSTULACION PARA PROYECTOS DE INTERCAMBIO ECOS...,ECOS200049,2020,2021,LAURA PEREZ MUNOZ,MUJER,INSTITUCION ASOCIADA,EUROPEAN SOUTHERN OBSERVATORY,-,ALEMANIA,REGION METROPOLITANA DE SANTIAGO,SREC,noaplica,False,EUROPEAN SOUTHERN OBSERVATORY


Revisamos ahora si queda alguna institución chilena con más de un RUT asociado.

In [ ]:
for name in df_anid.loc[df_anid["PAISINSTITUCION"] == "CHILE", "nombre_normalizado"].unique():
  ruts = df_anid.loc[df_anid["nombre_normalizado"] == name, "rut_normalizado"].unique()
  if len(ruts) != 1:
    print(name, ruts, len(ruts))

Todo bien. Revisemos cuantas filas tienen RUTs faltantes.

In [ ]:
df_anid[(~df_anid["rut_valido"]) & (df_anid["PAISINSTITUCION"] == "CHILE")]

,PROGRAMA,CONCURSO,FOLIO,AGNOPOSTULACION,AGNOFALLO,RESPONSABLEPROYECTO,SEXO,ROLINSTITUCION,INSTITUCION1ERNIVEL,RUTINSTITUCION,PAISINSTITUCION,REGIONINSTITUCION,SUBDIRECCION,rut_normalizado,rut_valido,nombre_normalizado
0,REGIONAL,II CONCURSO DE FORTALECIMIENTO A LA CONTINUIDA...,R15A10001,2015,2016,EDUARDO GRATACOS,HOMBRE,BENEFICIARIA PRINCIPAL,CENTRO REGIONAL DE INNOVACION HORTOFRUTICOLA ...,SIN INFORMACION,CHILE,REGION DE VALPARAISO,SCIA,-,False,CENTRO REGIONAL DE INNOVACION HORTOFRUTICOLA D...
1,REGIONAL,II CONCURSO DE FORTALECIMIENTO A LA CONTINUIDA...,R15A10002,2015,2016,LUIS ROJAS ARAYA,HOMBRE,BENEFICIARIA PRINCIPAL,CENTRO DE INVESTIGACION CIENTIFICO TECNOLOGICO...,SIN INFORMACION,CHILE,REGION DE ANTOFAGASTA,SCIA,-,False,CENTRO DE INVESTIGACION CIENTIFICO TECNOLOGICO...
3,REGIONAL,II CONCURSO DE FORTALECIMIENTO DE CENTROS REGI...,R16F10008,2016,2016,EDUARDO GRATACOS N.,HOMBRE,BENEFICIARIA PRINCIPAL,CENTRO REGIONAL DE INNOVACION HORTOFRUTICOLA ...,SIN INFORMACION,CHILE,REGION DE VALPARAISO,SCIA,-,False,CENTRO REGIONAL DE INNOVACION HORTOFRUTICOLA D...
4,REGIONAL,II CONCURSO DE FORTALECIMIENTO DE CENTROS REGI...,R16F10009,2016,2016,BERNARDO BROITMAN ROJAS,HOMBRE,BENEFICIARIA PRINCIPAL,CENTRO DE ESTUDIOS AVANZADOS EN ZONAS ARIDAS (...,SIN INFORMACION,CHILE,REGION DE COQUIMBO,SCIA,-,False,CENTRO DE ESTUDIOS AVANZADOS EN ZONAS ARIDAS (...
5,REGIONAL,II CONCURSO DE FORTALECIMIENTO DE CENTROS REGI...,R16F10002,2016,2016,CESAR BURGOS,HOMBRE,BENEFICIARIA PRINCIPAL,CENTRO DE GENOMICA NUTRICIONAL AGROACUICOLA (C...,SIN INFORMACION,CHILE,REGION DE LA ARAUCANIA,SCIA,-,False,CENTRO DE GENOMICA NUTRICIONAL AGROACUICOLA (C...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22954,PCI,FOMENTO A LA VINCULACION INTERNACIONAL PARA IN...,FOVI220124,2022,2022,MARISOL RUIZ CONTRERAS,MUJER,INSTITUCION ASOCIADA,ILUSTRE MUNICIPALIDAD DE VALDIVIA,-,CHILE,REGION DE LOS RIOS,SREC,-,False,ILUSTRE MUNICIPALIDAD DE VALDIVIA
22955,PCI,FOMENTO A LA VINCULACION INTERNACIONAL PARA IN...,FOVI220211,2022,2022,DANIEL MEDINA SALAS,HOMBRE,INSTITUCION ASOCIADA,SEREMI DE SALUD REGION DE LOS LAGOS,-,CHILE,NO REGISTRA REGION,SREC,-,False,SEREMI DE SALUD REGION DE LOS LAGOS
22956,PCI,FOMENTO A LA VINCULACION INTERNACIONAL PARA IN...,FOVI220211,2022,2022,DANIEL MEDINA SALAS,HOMBRE,INSTITUCION ASOCIADA,INSTITUTO NORUEGO DE INVESTIGACION DE AGUAS,-,CHILE,NO REGISTRA REGION,SREC,-,False,INSTITUTO NORUEGO DE INVESTIGACION DE AGUAS
23173,PCI,FOMENTO A LA VINCULACION INTERNACIONAL PARA IN...,FOVI220135,2022,2022,ARIEL LEIVA LOPEZ,HOMBRE,INSTITUCION EXTRANJERA,COMISION ECONOMICA PARA AMERICA LATINA Y EL CA...,NO APLICA,CHILE,REGION METROPOLITANA DE SANTIAGO,SREC,noaplica,False,COMISION ECONOMICA PARA AMERICA LATINA Y EL CA...


801 filas.

### Arreglo a partir de otras filas con nombres similares

Sabemos que pueden haber errores de tipeo, que hagan que dos nombres sean muy parecidos, pero no idénticos. Usaremos la distancia Levenshtein (o distancia de edición) para intentar encontrar esos casos

In [ ]:
names_without_rut = set(df_anid[(~df_anid["rut_valido"]) & (df_anid["PAISINSTITUCION"] == "CHILE")]["nombre_normalizado"].unique())
names_with_rut = set(df_anid[(df_anid["rut_valido"]) & (df_anid["PAISINSTITUCION"] == "CHILE")]["nombre_normalizado"].unique())

Notamos que la intersección entre ambos conjuntos es vacía (ya que arreglamos los casos de coincidencia de nombre exacto)

In [ ]:
names_without_rut & names_with_rut

set()

In [ ]:
unique_names = df_anid["nombre_normalizado"].unique()

min_distances = []
for name1 in names_without_rut:
  min_distance = 9999999
  min_name = ""
  for name2 in names_with_rut:
    current_distance = distance(name1, name2)
    if current_distance < min_distance:
      min_distance = current_distance
      min_name = name2
  min_distances.append([name1, min_name, min_distance])

In [ ]:
df_distances = pd.DataFrame(min_distances, columns=["Nombre", "Candidato", "Distancia"])
df_distances

,Nombre,Candidato,Distancia
0,ILUSTRE MUNICIPALIDAD DE SAN CLEMENTE,ILUSTRE MUNICIPALIDAD DE EL CARMEN,7
1,DPTO DE SALUD MUNICIPIO PADRE LAS CASAS,MUNICIPALIDAD DE PADRE LAS CASAS,20
2,PABLO CORREA OPAZO,AIKO LOGIC SPA,11
3,MINERA ANGLO AMERICAN CHILE LTDA,INPROIN CHILE LTDA,15
4,CENTRO DE RECURSOS HIDRICOS PARA LA AGRICULTUR...,INSTITUTO TECNOLOGICO PARA LA AGRICULTURA SUST...,29
...,...,...,...
437,RHONA SA,SONDA SA,3
438,CORPORACION AYUDA AL NINO LIMITADO,CORPORACION DE AYUDA AL NINO QUEMADO,8
439,INSTITUTO DE LA CONSTRUCCION,INSTITUTO DE DATA SCIENCE,12
440,FUNDACION AMERICA,FUNDACION MERI,3


In [ ]:
df_distances[df_distances["Distancia"] <= 5]

,Nombre,Candidato,Distancia
12,BASF CHILE SA,BOSCA CHILE SA,3
18,BIOGRAM SA,BIONET SA,4
19,MASISA SA,SIKA SA,3
32,ILUSTRE MUNICIPALIDAD DE PIRQUE,ILUSTRE MUNICIPALIDAD DE PITRUFQUEN,4
33,CIGIDEN,CINNDA,4
...,...,...,...
418,MUNICIPALIDAD DE TEMUCO,MUNICIPALIDAD DE PUCON,4
425,CADETECH S A,NANOTEC SA,5
427,SUMCITEC LTDA,QUIMTEC LTDA,4
437,RHONA SA,SONDA SA,3


Notamos que varios casos sí corresponden a errores de tipeo, mientras que otros son simplemente instituciones distintas con nombres relativamente similares.

In [ ]:
names_to_fix = ["EMPRESA NACIONAL DE TELECOMUNICACIONES",
                "SUBSECRETARIA DE MEDIO AMBIENTE",
                "UNIVERSIDAD DE O HIGGINS",
                "VIVEROS HIJUELAS SA",
                "ENGIE LABORELEC",
                "CORPORACION MUNICIPAL DE PUENTE ALTO",
                "NESTLE CHILE SA",
                "MARITIMA TRANSAUSTRAL LTDA",
                "ANGLOAMERICAN SUR SA",
                "SERVICIO DE SALUD ARAUCANIA SUR",
                "AGRICOLA GARCES LTDA",
                "UNIVERSIDAD BERNARDO OHIGGINS",
                "CENTRO INTERDISCIPLINARIO DE NEUROCIENCIA",
                "HENDRIX GENERICS AQUACULTURE SA",
                "CENTRO DE INVESTIGACION CIENTIFICO Y TECNOLOGICO PARA LA MINERIA",
                "CENTRO DE ESTUDIOS DE CONFLICTO Y COHESION SOCIALL",
                "CENTRO DE ESTUDIOS AVANZADOS EN ZONAS ARIDAS (CEAZA)",
                "CULTIVOS CHOLCHE"]

for name in names_to_fix:
  new_name = df_distances.loc[df_distances["Nombre"] == name, "Candidato"].values[0]
  ruts = df_anid.loc[(df_anid["nombre_normalizado"] == new_name), "rut_normalizado"].unique()
  assert len(ruts) == 1
  df_anid.loc[df_anid["nombre_normalizado"] == name,
              ["nombre_normalizado", "rut_normalizado", "rut_valido"]] = (new_name, ruts[0], True)

Revisamos que haya funcionado

In [ ]:
df_anid[df_anid["nombre_normalizado"] == "CENTRO DE ESTUDIOS DE CONFLICTO Y COHESION SOCIALL"]

,PROGRAMA,CONCURSO,FOLIO,AGNOPOSTULACION,AGNOFALLO,RESPONSABLEPROYECTO,SEXO,ROLINSTITUCION,INSTITUCION1ERNIVEL,RUTINSTITUCION,PAISINSTITUCION,REGIONINSTITUCION,SUBDIRECCION,rut_normalizado,rut_valido,nombre_normalizado


Ya no existen registros con ese nombre, lo cual es bueno.

In [ ]:
df_anid[df_anid["nombre_normalizado"] == "CENTRO DE ESTUDIOS DE CONFLICTO Y COHESION SOCIAL"]

,PROGRAMA,CONCURSO,FOLIO,AGNOPOSTULACION,AGNOFALLO,RESPONSABLEPROYECTO,SEXO,ROLINSTITUCION,INSTITUCION1ERNIVEL,RUTINSTITUCION,PAISINSTITUCION,REGIONINSTITUCION,SUBDIRECCION,rut_normalizado,rut_valido,nombre_normalizado
4757,FONDEF,CONCURSO IDEA I+D 2022,ID22I10009,2022,2022,LUIS DE LA VEGA RODRIGUEZ,HOMBRE,COLABORADORA,CENTRO DE ESTUDIOS DE CONFLICTO Y COHESION SOCIAL,12265668-3,CHILE,NO REGISTRA REGION,SIA,60910000-1,True,CENTRO DE ESTUDIOS DE CONFLICTO Y COHESION SOCIAL
6127,FONDECYT,FONDECYT REGULAR 2016,1160308,2015,2016,NICOLAS SOMMA GONZALEZ,HOMBRE,INSTITUCION SECUNDARIA,CENTRO DE ESTUDIOS DE CONFLICTO Y COHESION SOCIAL,-,NO REGISTRA PAIS,NO REGISTRA REGION,SPI,60910000-1,False,CENTRO DE ESTUDIOS DE CONFLICTO Y COHESION SOCIAL
6804,FONDECYT,FONDECYT REGULAR 2016,1161550,2015,2016,FELIPE LINK LAZO,HOMBRE,INSTITUCION APORTANTE,CENTRO DE ESTUDIOS DE CONFLICTO Y COHESION SOCIAL,-,NO REGISTRA PAIS,NO REGISTRA REGION,SPI,60910000-1,False,CENTRO DE ESTUDIOS DE CONFLICTO Y COHESION SOCIAL
13089,FONDECYT,FONDECYT DE INICIACION EN INVESTIGACION 2020,11201211,2020,2020,ANA MATEUS FIGUEIREDO,MUJER,INSTITUCION APORTANTE,CENTRO DE ESTUDIOS DE CONFLICTO Y COHESION SOCIAL,60910000-1,NO REGISTRA PAIS,NO REGISTRA REGION,SPI,60910000-1,True,CENTRO DE ESTUDIOS DE CONFLICTO Y COHESION SOCIAL
20170,PCI,CONCURSO DE APOYO A LA FORMACION DE REDES INTE...,REDI170399,2017,2017,ESTEBAN LOPEZ OCHOA,HOMBRE,INSTITUCION NACIONAL PATROCINANTE ASOCIADA,CENTRO DE ESTUDIOS DE CONFLICTO Y COHESION SOCIAL,SIN INFORMACION,NO REGISTRA PAIS,NO REGISTRA REGION,SREC,60910000-1,False,CENTRO DE ESTUDIOS DE CONFLICTO Y COHESION SOCIAL
20649,PCI,APOYO A LA FORMACION DE REDES INTERNACIONALES ...,REDES180048,2018,2018,NICOLAS SOMMA GONZALEZ,HOMBRE,CENTRO BENEFICIARIO,CENTRO DE ESTUDIOS DE CONFLICTO Y COHESION SOC...,SIN INFORMACION,CHILE,NO REGISTRA REGION,SREC,60910000-1,True,CENTRO DE ESTUDIOS DE CONFLICTO Y COHESION SOCIAL
21264,PCI,APOYO A LA FORMACION DE REDES INTERNACIONALES ...,REDES190042,2019,2019,DANTE CONTRERAS GUAJARDO,HOMBRE,CENTRO BENEFICIARIO,CENTRO DE ESTUDIOS DE CONFLICTO Y COHESION SOCIAL,60910000-1,NO REGISTRA PAIS,NO REGISTRA REGION,SREC,60910000-1,True,CENTRO DE ESTUDIOS DE CONFLICTO Y COHESION SOCIAL


Ahora tienen el nombre arreglado, junto al RUT válido asociado.

In [ ]:
df_anid[(~df_anid["rut_valido"]) & (df_anid["PAISINSTITUCION"] == "CHILE")]

,PROGRAMA,CONCURSO,FOLIO,AGNOPOSTULACION,AGNOFALLO,RESPONSABLEPROYECTO,SEXO,ROLINSTITUCION,INSTITUCION1ERNIVEL,RUTINSTITUCION,PAISINSTITUCION,REGIONINSTITUCION,SUBDIRECCION,rut_normalizado,rut_valido,nombre_normalizado
0,REGIONAL,II CONCURSO DE FORTALECIMIENTO A LA CONTINUIDA...,R15A10001,2015,2016,EDUARDO GRATACOS,HOMBRE,BENEFICIARIA PRINCIPAL,CENTRO REGIONAL DE INNOVACION HORTOFRUTICOLA ...,SIN INFORMACION,CHILE,REGION DE VALPARAISO,SCIA,-,False,CENTRO REGIONAL DE INNOVACION HORTOFRUTICOLA D...
1,REGIONAL,II CONCURSO DE FORTALECIMIENTO A LA CONTINUIDA...,R15A10002,2015,2016,LUIS ROJAS ARAYA,HOMBRE,BENEFICIARIA PRINCIPAL,CENTRO DE INVESTIGACION CIENTIFICO TECNOLOGICO...,SIN INFORMACION,CHILE,REGION DE ANTOFAGASTA,SCIA,-,False,CENTRO DE INVESTIGACION CIENTIFICO TECNOLOGICO...
3,REGIONAL,II CONCURSO DE FORTALECIMIENTO DE CENTROS REGI...,R16F10008,2016,2016,EDUARDO GRATACOS N.,HOMBRE,BENEFICIARIA PRINCIPAL,CENTRO REGIONAL DE INNOVACION HORTOFRUTICOLA ...,SIN INFORMACION,CHILE,REGION DE VALPARAISO,SCIA,-,False,CENTRO REGIONAL DE INNOVACION HORTOFRUTICOLA D...
5,REGIONAL,II CONCURSO DE FORTALECIMIENTO DE CENTROS REGI...,R16F10002,2016,2016,CESAR BURGOS,HOMBRE,BENEFICIARIA PRINCIPAL,CENTRO DE GENOMICA NUTRICIONAL AGROACUICOLA (C...,SIN INFORMACION,CHILE,REGION DE LA ARAUCANIA,SCIA,-,False,CENTRO DE GENOMICA NUTRICIONAL AGROACUICOLA (C...
6,REGIONAL,II CONCURSO DE FORTALECIMIENTO DE CENTROS REGI...,R16F10013,2016,2016,DANIELA NUÃ±EZ BERNAL,MUJER,BENEFICIARIA PRINCIPAL,CENTRO DE INVESTIGACION EN POLIMEROS AVANZADOS...,SIN INFORMACION,CHILE,REGION DEL BIO-BIO,SCIA,-,False,CENTRO DE INVESTIGACION EN POLIMEROS AVANZADOS...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22954,PCI,FOMENTO A LA VINCULACION INTERNACIONAL PARA IN...,FOVI220124,2022,2022,MARISOL RUIZ CONTRERAS,MUJER,INSTITUCION ASOCIADA,ILUSTRE MUNICIPALIDAD DE VALDIVIA,-,CHILE,REGION DE LOS RIOS,SREC,-,False,ILUSTRE MUNICIPALIDAD DE VALDIVIA
22955,PCI,FOMENTO A LA VINCULACION INTERNACIONAL PARA IN...,FOVI220211,2022,2022,DANIEL MEDINA SALAS,HOMBRE,INSTITUCION ASOCIADA,SEREMI DE SALUD REGION DE LOS LAGOS,-,CHILE,NO REGISTRA REGION,SREC,-,False,SEREMI DE SALUD REGION DE LOS LAGOS
22956,PCI,FOMENTO A LA VINCULACION INTERNACIONAL PARA IN...,FOVI220211,2022,2022,DANIEL MEDINA SALAS,HOMBRE,INSTITUCION ASOCIADA,INSTITUTO NORUEGO DE INVESTIGACION DE AGUAS,-,CHILE,NO REGISTRA REGION,SREC,-,False,INSTITUTO NORUEGO DE INVESTIGACION DE AGUAS
23173,PCI,FOMENTO A LA VINCULACION INTERNACIONAL PARA IN...,FOVI220135,2022,2022,ARIEL LEIVA LOPEZ,HOMBRE,INSTITUCION EXTRANJERA,COMISION ECONOMICA PARA AMERICA LATINA Y EL CA...,NO APLICA,CHILE,REGION METROPOLITANA DE SANTIAGO,SREC,noaplica,False,COMISION ECONOMICA PARA AMERICA LATINA Y EL CA...


779 filas faltantes.

### Imputación a partir de API externa

In [ ]:
df_inst_without_ruts = df_anid[(~df_anid["rut_valido"]) & (df_anid["PAISINSTITUCION"] == "CHILE")] \
                        .groupby("nombre_normalizado", as_index=False)["FOLIO"] \
                        .count()\
                        .rename(columns={"FOLIO": "N filas"})\
                        .sort_values("N filas")
df_inst_without_ruts

,nombre_normalizado,N filas
0,ACADEMIA CHILENA DE LA LENGUA,1
282,LABORATORIO XIMENA POLANCO,1
281,LABORATORIO NOVARTIS,1
280,LABORATORIO DE ECOLOGIA Y PAISAJE / FACULTAD D...,1
279,LABORATORIO DE DENDROCRONOLOGIA Y CAMBIO GLOBA...,1
...,...,...
217,ILUSTRE MUNICIPALIDAD DE CHILLAN,4
205,GOBIERNO REGIONAL REGION METROPOLITANA,5
380,SIN INSTITUCION,5
290,MASISA SA,5


Intentaremos imputar estos RUTs a partir de datos de internet. Para eso, usaremos la API de [Portal Chile](https://www.portalchile.org).

#### Descarga de RUTs sugeridos desde API

In [ ]:
def get_suggested_rut(name):
  url = f"https://www.portalchile.org/api/suggest/{name}?opcion=E"
  headers = {
      "User-Agent":"Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36"
    }

  response = requests.get(url, headers=headers)
  if response.status_code != 200:
    return {"rut": None, "nombre": None}
  suggestions = response.json()["sugerencia"]
  if not suggestions:
    return {"rut": None, "nombre": None}

  first_suggestion = suggestions[0]["_source"]
  suggested_name = first_suggestion["razon_social"]
  suggested_rut = first_suggestion["rut"] + "-" + first_suggestion["dv"]
  return {"rut": suggested_rut, "nombre": suggested_name}

In [ ]:
results_api = {}
with concurrent.futures.ThreadPoolExecutor(max_workers=10) as executor:
  futures = {
    executor.submit(get_suggested_rut, name): name
    for name in df_inst_without_ruts["nombre_normalizado"].unique()
  }
  for future in tqdm(concurrent.futures.as_completed(futures)):
    name = futures[future]
    results_api[name] = future.result()

424it [00:55,  7.71it/s]


In [ ]:
df_suggested_ruts = pd.DataFrame.from_dict(results_api, orient="index")\
                    .reset_index()\
                    .rename(columns={"index": "Nombre original",
                                     "rut": "Rut sugerido",
                                     "nombre": "Nombre sugerido"})
df_suggested_ruts


,Nombre original,Rut sugerido,Nombre sugerido
0,LABORATORIO DE CONVERSION DE ENERGIAS Y ELECTR...,76609531-3,ELECTRONICA DE POTENCIA LUIS MELLA SPA
1,LABORATORIO DE ECOLOGIA Y PAISAJE / FACULTAD D...,65134563-4,CORPORACION DE ECOLOGIA SOCIAL Y ECOLOGIA POLI...
2,ACADEMIA CHILENA DE LA LENGUA,72434600-6,ACADEMIA CHILENA DE CIENCIAS
3,LABORATORIO DE ANALISIS E INVESTIGACIONES ARQU...,78847500-4,ANALISIS LABORATORIO LTDA
4,LABORATORIO XIMENA POLANCO,76021365-9,LABORATORIO XIMENA GONZALEZ LIMITADA
...,...,...,...
419,GOBIERNO REGIONAL REGION METROPOLITANA,61923200-3,GOBIERNO REGIONAL REGION METROPOLITANA
420,SIN INSTITUCION,76974403-7,SIN SPA
421,MASISA SA,96802690-9,MASISA S.A.
422,PARTICULAR,50261240-9,ESCUELA PARTICULAR LA FRONTERA LIMITADA


Guardaremos este dataframe, por si acaso 👀

In [ ]:
df_suggested_ruts.to_excel(os.path.join(results_dir, "data_ruts_sugeridos.xlsx"), index=False)

#### Análisis de RUTs sugeridos

Ahora procederemos al análisis. Lo primero que haremos será reemplazar los nombres nulos por string vacíos (eso hará más fácil la manipulación luego)

In [ ]:
df_suggested_ruts.loc[df_suggested_ruts["Nombre sugerido"].isnull(), "Nombre sugerido"] = ""

Ahora normalizaremos el nombre sugerido tal como lo hicimos con los nombres de las instituciones.

In [ ]:
df_suggested_ruts["Nombre sugerido"] = df_suggested_ruts["Nombre sugerido"].str.upper().str.strip().str.replace(".", "").str.replace("  ", " ").apply(unidecode)
df_suggested_ruts

<ipython-input-62-4d7a89b37786>:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df_suggested_ruts["Nombre sugerido"] = df_suggested_ruts["Nombre sugerido"].str.upper().str.strip().str.replace(".", "").str.replace("  ", " ").apply(unidecode)


,Nombre original,Rut sugerido,Nombre sugerido
0,LABORATORIO DE CONVERSION DE ENERGIAS Y ELECTR...,76609531-3,ELECTRONICA DE POTENCIA LUIS MELLA SPA
1,LABORATORIO DE ECOLOGIA Y PAISAJE / FACULTAD D...,65134563-4,CORPORACION DE ECOLOGIA SOCIAL Y ECOLOGIA POLI...
2,ACADEMIA CHILENA DE LA LENGUA,72434600-6,ACADEMIA CHILENA DE CIENCIAS
3,LABORATORIO DE ANALISIS E INVESTIGACIONES ARQU...,78847500-4,ANALISIS LABORATORIO LTDA
4,LABORATORIO XIMENA POLANCO,76021365-9,LABORATORIO XIMENA GONZALEZ LIMITADA
...,...,...,...
419,GOBIERNO REGIONAL REGION METROPOLITANA,61923200-3,GOBIERNO REGIONAL REGION METROPOLITANA
420,SIN INSTITUCION,76974403-7,SIN SPA
421,MASISA SA,96802690-9,MASISA SA
422,PARTICULAR,50261240-9,ESCUELA PARTICULAR LA FRONTERA LIMITADA


Añadiremos además una columna `Verificado`, que indicará si el RUT sugerido es correcto.

In [ ]:
df_suggested_ruts["Verificado"] = False

Veamos primero los casos en que el nombre sugerido es idéntico al nombre original.

In [ ]:
df_suggested_ruts[df_suggested_ruts["Nombre original"] == df_suggested_ruts["Nombre sugerido"] ]

,Nombre original,Rut sugerido,Nombre sugerido,Verificado
11,KNOP LABORATORIOS S A,89688800-5,KNOP LABORATORIOS S A,False
30,MAQUI NEW LIFE SA,76128685-4,MAQUI NEW LIFE SA,False
33,MINISTERIO PUBLICO,61935400-1,MINISTERIO PUBLICO,False
35,MINISTERIO DE BIENES NACIONALES,61402000-8,MINISTERIO DE BIENES NACIONALES,False
54,ILUSTRE MUNICIPALIDAD DE MACUL,69253700-9,ILUSTRE MUNICIPALIDAD DE MACUL,False
78,INSTITUTO DE LA CONSTRUCCION,73116100-3,INSTITUTO DE LA CONSTRUCCION,False
93,INSTITUTO NACIONAL DE LA JUVENTUD,60110000-2,INSTITUTO NACIONAL DE LA JUVENTUD,False
96,FUNDACION PARENTESIS,72421000-7,FUNDACION PARENTESIS,False
105,SOCIEDAD CHILENA DE ALERGIA E INMUNOLOGIA,70577200-2,SOCIEDAD CHILENA DE ALERGIA E INMUNOLOGIA,False
109,SOC DE INSTRUCCION PRIMARIA DE SANTIAGO,82648400-4,SOC DE INSTRUCCION PRIMARIA DE SANTIAGO,False


En todos estos casos, el RUT sugerido es correcto, así que colocaremos True en la columna `Verificado`

In [ ]:
df_suggested_ruts.loc[df_suggested_ruts["Nombre original"] == df_suggested_ruts["Nombre sugerido"], "Verificado"] = True

Veamos cuántos casos verificados llevamos y cuántos nos faltan

In [ ]:
print("Verificados", len(df_suggested_ruts[df_suggested_ruts["Verificado"]]))
print("No verificados", len(df_suggested_ruts[~df_suggested_ruts["Verificado"]]))

Verificados 43
No verificados 381


Analicemos los casos usando la distancia de edición.

In [ ]:
df_suggested_ruts["Distancia"] = df_suggested_ruts.apply(lambda row: distance(row["Nombre original"], row["Nombre sugerido"]), axis=1)
df_suggested_ruts

,Nombre original,Rut sugerido,Nombre sugerido,Verificado,Distancia
0,LABORATORIO DE CONVERSION DE ENERGIAS Y ELECTR...,76609531-3,ELECTRONICA DE POTENCIA LUIS MELLA SPA,False,51
1,LABORATORIO DE ECOLOGIA Y PAISAJE / FACULTAD D...,65134563-4,CORPORACION DE ECOLOGIA SOCIAL Y ECOLOGIA POLI...,False,65
2,ACADEMIA CHILENA DE LA LENGUA,72434600-6,ACADEMIA CHILENA DE CIENCIAS,False,7
3,LABORATORIO DE ANALISIS E INVESTIGACIONES ARQU...,78847500-4,ANALISIS LABORATORIO LTDA,False,41
4,LABORATORIO XIMENA POLANCO,76021365-9,LABORATORIO XIMENA GONZALEZ LIMITADA,False,15
...,...,...,...,...,...
419,GOBIERNO REGIONAL REGION METROPOLITANA,61923200-3,GOBIERNO REGIONAL REGION METROPOLITANA,True,0
420,SIN INSTITUCION,76974403-7,SIN SPA,False,10
421,MASISA SA,96802690-9,MASISA SA,True,0
422,PARTICULAR,50261240-9,ESCUELA PARTICULAR LA FRONTERA LIMITADA,False,29


Sólo para estar seguros, revisaremos que los casos donde la distancia es cero ya tengan verificado=True (por el análisis previo)

In [ ]:
df_suggested_ruts.loc[df_suggested_ruts["Distancia"] == 0, "Verificado"].all()

True

Todo bien. Enfoquémonos ahora en los que tienen distancia mayor a cero y menor a cierto umbral (por ahora, 5)

In [ ]:
df_suggested_ruts[(df_suggested_ruts["Distancia"] > 0) & (df_suggested_ruts["Distancia"] < 5)]

,Nombre original,Rut sugerido,Nombre sugerido,Verificado,Distancia
16,INVERSIONES Y SERVICIOS ABUD SITTLER LTDA,76202636-8,INVERSIONES Y SERVICIOS ABUD SITTLER LIMITADA,False,4
19,INSTITUTO TECNOLOGICO PARA LA AGRICULTURA SUST...,96948520-6,INSTITUTO TECNOLOGICO PARA LA AGRICULTURA SUST...,False,1
26,MUNICIPALIDAD DE LA GRANJA,69072400-6,I MUNICIPALIDAD DE LA GRANJA,False,2
32,MODULOS WEWFE LTDA,76156532-K,MODULOS WEWFE LIMITADA,False,4
38,MINERA ANGLO AMERICAN CHILE LTDA,87522100-0,MINERA ANGLO AMERICAN CHILE LIMITADA,False,4
55,ILUSTRE MUNICIPALIDAD DE ARICA,69010200-5,ILUSTRE MUNICIPALIDAD DE HUARA,False,4
57,ILUSTRE MUNICIPALIDAD DE HUALQUI,69010200-5,ILUSTRE MUNICIPALIDAD DE HUARA,False,4
95,ILUSTRE MUNICIPALIDAD DE TALCA,69010200-5,ILUSTRE MUNICIPALIDAD DE HUARA,False,4
102,SOCIEDAD CHILENA DE MEDICINA FISICA Y REHABILT...,70850700-8,SOCIEDAD CHILENA DE MEDICINA FISICA Y REHABILI...,False,1
103,N-ACTIVE EIRL,76066679-3,N-ACTIVE SPA,False,4


La gran mayoría de los casos sí corresponden a la misma institución. Sólo son expceciones las siguientes:

- ILUSTRE MUNICIPALIDAD DE HUALQUI
- ILUSTRE MUNICIPALIDAD DE ARICA
- ILUSTRE MUNICIPALIDAD DE TALCA
- UNIVERSIDAD DE LA CORUNA

En estos casos, indicaremos que la distancia de edición es 9999, un valor muy alto para señalar que efectivamente están muy lejos.

In [ ]:
wrong_suggestions = ["ILUSTRE MUNICIPALIDAD DE HUALQUI",
                    "ILUSTRE MUNICIPALIDAD DE ARICA",
                    "ILUSTRE MUNICIPALIDAD DE TALCA",
                    "UNIVERSIDAD DE LA CORUNA"]
df_suggested_ruts.loc[df_suggested_ruts["Nombre original"].isin(wrong_suggestions), "Distancia"] = 9999
df_suggested_ruts.loc[df_suggested_ruts["Nombre original"].isin(wrong_suggestions)]

,Nombre original,Rut sugerido,Nombre sugerido,Verificado,Distancia
55,ILUSTRE MUNICIPALIDAD DE ARICA,69010200-5,ILUSTRE MUNICIPALIDAD DE HUARA,False,9999
57,ILUSTRE MUNICIPALIDAD DE HUALQUI,69010200-5,ILUSTRE MUNICIPALIDAD DE HUARA,False,9999
95,ILUSTRE MUNICIPALIDAD DE TALCA,69010200-5,ILUSTRE MUNICIPALIDAD DE HUARA,False,9999
372,UNIVERSIDAD DE LA CORUNA,70783100-6,UNIVERSIDAD DE LA SERENA,False,9999


Para todos los otros casos con distancia menor a 5, las sugerencias son correctas, por lo cual pondremos True en la columna Verificado.

In [ ]:
df_suggested_ruts.loc[(df_suggested_ruts["Distancia"] > 0) & (df_suggested_ruts["Distancia"] < 5), "Verificado"] = True

Veamos ahora cuántos casos llevamos.

In [ ]:
print("Verificados", len(df_suggested_ruts[df_suggested_ruts["Verificado"]]))
print("No verificados", len(df_suggested_ruts[~df_suggested_ruts["Verificado"]]))

Verificados 70
No verificados 354


Fijémonos ahora en los casos con distancia mayor a 5 y menor a 10.

In [ ]:
df_suggested_ruts[(df_suggested_ruts["Distancia"] > 5) & (df_suggested_ruts["Distancia"] <= 10)]

,Nombre original,Rut sugerido,Nombre sugerido,Verificado,Distancia
2,ACADEMIA CHILENA DE LA LENGUA,72434600-6,ACADEMIA CHILENA DE CIENCIAS,False,7
7,LABORATORIO NOVARTIS,76330711-5,LABORATORIO FRULAB SPA,False,9
13,INTENDENCIA REGIONAL DE AYSEN,72222000-5,GOBIERNO REGIONAL DE AYSEN,False,10
18,JUNTA DE VIGILANCIA DE LA SEGUNDA SECCION DEL ...,73703300-7,JUNTA DE VIGILANCIA DE LA SEGUNDA SECCION DEL ...,False,7
22,MUNICIPALIDAD DE COPIAPO,69040700-0,MUNICIPALIDAD DE OVALLE,False,6
...,...,...,...,...,...
411,SOCIEDAD CHILENA DE PEDIATRIA,65020003-9,SOCIEDAD CHILENA DE ESTADISTICA,False,7
412,ILUSTRE MUNICIPALIDAD DE TEMUCO,69010200-5,ILUSTRE MUNICIPALIDAD DE HUARA,False,6
416,CENTRO DE ESTUDIOS INTERCULTURALES E INDIGENAS,77298200-3,CENTRO DE ESTUDIOS INTERCULTURALES LIMITADA,False,8
417,ILUSTRE MUNICIPALIDAD DE VALDIVIA,69010200-5,ILUSTRE MUNICIPALIDAD DE HUARA,False,7


A diferencia del caso anterior, ahora sólo algunas pocas sugerencias son correctas. A saber:
- LACTEOS WINKLER LIMITADA
- MARINE HARVEST CHILE SA
- INSTITUTO NACIONAL DE DERECHOS HUMANOS (INDH)
- SOCIEDAD AGRICOLA SANTA MARIA EIRL
- SEIGARD S A
- FUNDACION ESTUDIO PARA UN HERMANO EDUCERE
- CENTRO DE INVESTIGACIONES DEL HOMBRE EN EL DESIERTO (CIHDE)
- CENTRO REGIONAL DE ESTUDIOS EN ALIMENTOS SALUDABLES (CREAS)
- ASYMA AG
- CENTRO ARBOL SPA
- CENTRAL UNITARIA DE TRABAJADORES
- CENTRO DE ENVASES Y EMBALAJES DE CHILE CENEM
- DMT GMBH & CO
- ASOCIACION DE EXPORTADORES DE FRUTAS DE CHILE AG ( ASOEX)
- FUNDACION TELEFONICA
- EMPRESA NACIONAL DE MINERIA - ENAMI
- CENTRO DE DIAGNOSTICO VETERINARIO SA
- CENTRO DE INVESTIGACION CIENTIFICO TECNOLOGICO PARA LA MINERIA (CICITEM)


In [ ]:
valid_suggestions = ["LACTEOS WINKLER LIMITADA",
                    "MARINE HARVEST CHILE SA",
                    "INSTITUTO NACIONAL DE DERECHOS HUMANOS (INDH)",
                    "SOCIEDAD AGRICOLA SANTA MARIA EIRL",
                    "SEIGARD S A",
                    "FUNDACION ESTUDIO PARA UN HERMANO EDUCERE",
                    "CENTRO DE INVESTIGACIONES DEL HOMBRE EN EL DESIERTO (CIHDE)",
                    "CENTRO REGIONAL DE ESTUDIOS EN ALIMENTOS SALUDABLES (CREAS)",
                    "ASYMA AG",
                    "CENTRO ARBOL SPA",
                    "CENTRAL UNITARIA DE TRABAJADORES",
                    "CENTRO DE ENVASES Y EMBALAJES DE CHILE CENEM",
                    "DMT GMBH & CO",
                    "ASOCIACION DE EXPORTADORES DE FRUTAS DE CHILE AG ( ASOEX)",
                    "FUNDACION TELEFONICA",
                    "EMPRESA NACIONAL DE MINERIA - ENAMI",
                    "CENTRO DE DIAGNOSTICO VETERINARIO SA",
                    "CENTRO DE INVESTIGACION CIENTIFICO TECNOLOGICO PARA LA MINERIA (CICITEM)"]
df_suggested_ruts.loc[df_suggested_ruts["Nombre original"].isin(valid_suggestions), "Verificado"] = True

Notamos que ya no tiene mucho sentido seguir con el análisis para umbrales mayores; la distancia es cada vez mayor, y eso suele indicar que sí corresponde a instituciones distintas. Revisaremos finalmente cuántos RUTs pudimos rescatar.

In [ ]:
print("Verificados", len(df_suggested_ruts[df_suggested_ruts["Verificado"]]))
print("No verificados", len(df_suggested_ruts[~df_suggested_ruts["Verificado"]]))

Verificados 88
No verificados 336


#### Ingresar RUTs a dataframe base

Primero, eliminaremos los ruts nulos y los reemplazaremos por "-".

In [ ]:
df_anid.loc[df_anid["rut_normalizado"].isnull(), "rut_normalizado"] = "-"

Primero, combinaremos la base de datos de ANID (`df_anid`) con la base de datos de RUTs que acabamos de crear (`df_suggested_ruts`).

In [ ]:
df_merged = df_anid.merge(df_suggested_ruts[df_suggested_ruts["Verificado"]], how="left", left_on="nombre_normalizado", right_on="Nombre original")
df_merged

,PROGRAMA,CONCURSO,FOLIO,AGNOPOSTULACION,AGNOFALLO,RESPONSABLEPROYECTO,SEXO,ROLINSTITUCION,INSTITUCION1ERNIVEL,RUTINSTITUCION,...,REGIONINSTITUCION,SUBDIRECCION,rut_normalizado,rut_valido,nombre_normalizado,Nombre original,Rut sugerido,Nombre sugerido,Verificado,Distancia
0,REGIONAL,II CONCURSO DE FORTALECIMIENTO A LA CONTINUIDA...,R15A10001,2015,2016,EDUARDO GRATACOS,HOMBRE,BENEFICIARIA PRINCIPAL,CENTRO REGIONAL DE INNOVACION HORTOFRUTICOLA ...,SIN INFORMACION,...,REGION DE VALPARAISO,SCIA,-,False,CENTRO REGIONAL DE INNOVACION HORTOFRUTICOLA D...,NaN,NaN,NaN,NaN,NaN
1,REGIONAL,II CONCURSO DE FORTALECIMIENTO A LA CONTINUIDA...,R15A10002,2015,2016,LUIS ROJAS ARAYA,HOMBRE,BENEFICIARIA PRINCIPAL,CENTRO DE INVESTIGACION CIENTIFICO TECNOLOGICO...,SIN INFORMACION,...,REGION DE ANTOFAGASTA,SCIA,-,False,CENTRO DE INVESTIGACION CIENTIFICO TECNOLOGICO...,CENTRO DE INVESTIGACION CIENTIFICO TECNOLOGICO...,65937660-1,CENTRO DE INVESTIGACION CIENTIFICO TECNOLOGICO...,True,10.0
2,PIA,CONCURSO NACIONAL DE ASIGNACION DE TIEMPO DE B...,AUB160003,2016,2016,RUBEN ESCRIBANO VELOSO,HOMBRE,INSTITUCION PATROCINANTE,UNIVERSIDAD DE CONCEPCION,81494400-K,...,REGION DEL BIO-BIO,SCIA,81494400-k,True,UNIVERSIDAD DE CONCEPCION,NaN,NaN,NaN,NaN,NaN
3,REGIONAL,II CONCURSO DE FORTALECIMIENTO DE CENTROS REGI...,R16F10008,2016,2016,EDUARDO GRATACOS N.,HOMBRE,BENEFICIARIA PRINCIPAL,CENTRO REGIONAL DE INNOVACION HORTOFRUTICOLA ...,SIN INFORMACION,...,REGION DE VALPARAISO,SCIA,-,False,CENTRO REGIONAL DE INNOVACION HORTOFRUTICOLA D...,NaN,NaN,NaN,NaN,NaN
4,REGIONAL,II CONCURSO DE FORTALECIMIENTO DE CENTROS REGI...,R16F10009,2016,2016,BERNARDO BROITMAN ROJAS,HOMBRE,BENEFICIARIA PRINCIPAL,CENTRO DE ESTUDIOS AVANZADOS EN ZONAS ARIDAS (...,SIN INFORMACION,...,REGION DE COQUIMBO,SCIA,65997470-3,True,CENTRO DE ESTUDIOS AVANZADOS EN ZONAS ARIDAS -...,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23703,PCI,PROGRAMAS REGIONALES STIC-AMSUD - MATH-AMSUD -...,AMSUD220044,2022,2023,DENISSE MUÑANTE ARZAPALO,FEMENINO,INSTITUCION DE FRANCIA,ECOLE NATIONALE SUPERIEURE D’INFORMATIQUE POUR...,NaN,...,NaN,SREC,-,False,ECOLE NATIONALE SUPERIEURE D'INFORMATIQUE POUR...,NaN,NaN,NaN,NaN,NaN
23704,PCI,PROGRAMAS REGIONALES STIC-AMSUD - MATH-AMSUD -...,AMSUD220045,2022,2023,HUGO CARRILLO LINCOPI,MASCULINO,INSTITUCION DE FRANCIA,INRIA PARIS,NaN,...,NaN,SREC,-,False,INRIA PARIS,NaN,NaN,NaN,NaN,NaN
23705,PCI,PROGRAMAS REGIONALES STIC-AMSUD - MATH-AMSUD -...,AMSUD220045,2022,2023,HUGO CARRILLO LINCOPI,MASCULINO,INSTITUCION SUDAMERICANA,ESCUELA POLITECNICA NACIONAL,NaN,...,NaN,SREC,-,False,ESCUELA POLITECNICA NACIONAL,NaN,NaN,NaN,NaN,NaN
23706,PCI,PROGRAMAS REGIONALES STIC-AMSUD - MATH-AMSUD -...,AMSUD220045,2022,2023,HUGO CARRILLO LINCOPI,MASCULINO,INSTITUCION SUDAMERICANA,FUNDACION INRIA CHILE,65058044-3,...,NaN,SREC,65058044-3,True,FUNDACION INRIA CHILE,NaN,NaN,NaN,NaN,NaN


In [ ]:
def combine_ruts(rut1, rut2):
  if rut2 is None:
    return rut1
  if rut1 is None or rut1 == "-" or rut1 == "noaplica":
    return rut2
  return rut1

df_merged["rut_combinado"] = df_merged["rut_normalizado"].combine(df_merged["Rut sugerido"], combine_ruts)
df_merged

,PROGRAMA,CONCURSO,FOLIO,AGNOPOSTULACION,AGNOFALLO,RESPONSABLEPROYECTO,SEXO,ROLINSTITUCION,INSTITUCION1ERNIVEL,RUTINSTITUCION,...,SUBDIRECCION,rut_normalizado,rut_valido,nombre_normalizado,Nombre original,Rut sugerido,Nombre sugerido,Verificado,Distancia,rut_combinado
0,REGIONAL,II CONCURSO DE FORTALECIMIENTO A LA CONTINUIDA...,R15A10001,2015,2016,EDUARDO GRATACOS,HOMBRE,BENEFICIARIA PRINCIPAL,CENTRO REGIONAL DE INNOVACION HORTOFRUTICOLA ...,SIN INFORMACION,...,SCIA,-,False,CENTRO REGIONAL DE INNOVACION HORTOFRUTICOLA D...,NaN,NaN,NaN,NaN,NaN,NaN
1,REGIONAL,II CONCURSO DE FORTALECIMIENTO A LA CONTINUIDA...,R15A10002,2015,2016,LUIS ROJAS ARAYA,HOMBRE,BENEFICIARIA PRINCIPAL,CENTRO DE INVESTIGACION CIENTIFICO TECNOLOGICO...,SIN INFORMACION,...,SCIA,-,False,CENTRO DE INVESTIGACION CIENTIFICO TECNOLOGICO...,CENTRO DE INVESTIGACION CIENTIFICO TECNOLOGICO...,65937660-1,CENTRO DE INVESTIGACION CIENTIFICO TECNOLOGICO...,True,10.0,65937660-1
2,PIA,CONCURSO NACIONAL DE ASIGNACION DE TIEMPO DE B...,AUB160003,2016,2016,RUBEN ESCRIBANO VELOSO,HOMBRE,INSTITUCION PATROCINANTE,UNIVERSIDAD DE CONCEPCION,81494400-K,...,SCIA,81494400-k,True,UNIVERSIDAD DE CONCEPCION,NaN,NaN,NaN,NaN,NaN,81494400-k
3,REGIONAL,II CONCURSO DE FORTALECIMIENTO DE CENTROS REGI...,R16F10008,2016,2016,EDUARDO GRATACOS N.,HOMBRE,BENEFICIARIA PRINCIPAL,CENTRO REGIONAL DE INNOVACION HORTOFRUTICOLA ...,SIN INFORMACION,...,SCIA,-,False,CENTRO REGIONAL DE INNOVACION HORTOFRUTICOLA D...,NaN,NaN,NaN,NaN,NaN,NaN
4,REGIONAL,II CONCURSO DE FORTALECIMIENTO DE CENTROS REGI...,R16F10009,2016,2016,BERNARDO BROITMAN ROJAS,HOMBRE,BENEFICIARIA PRINCIPAL,CENTRO DE ESTUDIOS AVANZADOS EN ZONAS ARIDAS (...,SIN INFORMACION,...,SCIA,65997470-3,True,CENTRO DE ESTUDIOS AVANZADOS EN ZONAS ARIDAS -...,NaN,NaN,NaN,NaN,NaN,65997470-3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23703,PCI,PROGRAMAS REGIONALES STIC-AMSUD - MATH-AMSUD -...,AMSUD220044,2022,2023,DENISSE MUÑANTE ARZAPALO,FEMENINO,INSTITUCION DE FRANCIA,ECOLE NATIONALE SUPERIEURE D’INFORMATIQUE POUR...,NaN,...,SREC,-,False,ECOLE NATIONALE SUPERIEURE D'INFORMATIQUE POUR...,NaN,NaN,NaN,NaN,NaN,NaN
23704,PCI,PROGRAMAS REGIONALES STIC-AMSUD - MATH-AMSUD -...,AMSUD220045,2022,2023,HUGO CARRILLO LINCOPI,MASCULINO,INSTITUCION DE FRANCIA,INRIA PARIS,NaN,...,SREC,-,False,INRIA PARIS,NaN,NaN,NaN,NaN,NaN,NaN
23705,PCI,PROGRAMAS REGIONALES STIC-AMSUD - MATH-AMSUD -...,AMSUD220045,2022,2023,HUGO CARRILLO LINCOPI,MASCULINO,INSTITUCION SUDAMERICANA,ESCUELA POLITECNICA NACIONAL,NaN,...,SREC,-,False,ESCUELA POLITECNICA NACIONAL,NaN,NaN,NaN,NaN,NaN,NaN
23706,PCI,PROGRAMAS REGIONALES STIC-AMSUD - MATH-AMSUD -...,AMSUD220045,2022,2023,HUGO CARRILLO LINCOPI,MASCULINO,INSTITUCION SUDAMERICANA,FUNDACION INRIA CHILE,65058044-3,...,SREC,65058044-3,True,FUNDACION INRIA CHILE,NaN,NaN,NaN,NaN,NaN,65058044-3


Revisaremos que efectivamente la combinación que acabamos de realizar haya sido correcta. Para ello, revisaremos que el conjunto de nombres con RUTs válidos (provenientes de `df_suggested_ruts`) sea idéntico al conjunto de nombres de instituciones con RUTs que fueron actualizados.

In [ ]:
suggested_names = df_suggested_ruts.loc[df_suggested_ruts["Verificado"], "Nombre original"].unique()
names_rut_replaced = df_merged[(df_merged["rut_normalizado"] != df_merged["rut_combinado"])
                                & (~df_merged["rut_combinado"].isnull())]["nombre_normalizado"].unique()

assert set(suggested_names) == set(names_rut_replaced)

Todo bien :)

Normalizaremos los RUTs y volveremos a calcular su validez.

In [ ]:
df_merged["rut_combinado"] = df_merged["rut_combinado"].str.lower()
df_merged["rut_valido_2"] = df_merged["rut_combinado"].apply(valid_rut_wrapper)


Revisaremos que no haya ningún caso de un RUT que antes haya sido válido y ahora no.

In [ ]:
df_merged[(df_merged["rut_valido"]) & (~df_merged["rut_valido_2"])]

,PROGRAMA,CONCURSO,FOLIO,AGNOPOSTULACION,AGNOFALLO,RESPONSABLEPROYECTO,SEXO,ROLINSTITUCION,INSTITUCION1ERNIVEL,RUTINSTITUCION,...,rut_normalizado,rut_valido,nombre_normalizado,Nombre original,Rut sugerido,Nombre sugerido,Verificado,Distancia,rut_combinado,rut_valido_2


Todo bien. Ahora eliminaremos las columnas sobrantes

In [ ]:
df_merged = df_merged.drop(columns=["Nombre original", "Rut sugerido", "Nombre sugerido", "Verificado", "Distancia", "rut_normalizado", "rut_valido"])\
                     .rename(columns={"rut_combinado": "rut_normalizado",
                                      "rut_valido_2": "rut_valido"})
df_merged.loc[df_merged["rut_normalizado"].isnull(), "rut_normalizado"] = "-"
df_anid = df_merged
df_anid

,PROGRAMA,CONCURSO,FOLIO,AGNOPOSTULACION,AGNOFALLO,RESPONSABLEPROYECTO,SEXO,ROLINSTITUCION,INSTITUCION1ERNIVEL,RUTINSTITUCION,PAISINSTITUCION,REGIONINSTITUCION,SUBDIRECCION,nombre_normalizado,rut_normalizado,rut_valido
0,REGIONAL,II CONCURSO DE FORTALECIMIENTO A LA CONTINUIDA...,R15A10001,2015,2016,EDUARDO GRATACOS,HOMBRE,BENEFICIARIA PRINCIPAL,CENTRO REGIONAL DE INNOVACION HORTOFRUTICOLA ...,SIN INFORMACION,CHILE,REGION DE VALPARAISO,SCIA,CENTRO REGIONAL DE INNOVACION HORTOFRUTICOLA D...,-,False
1,REGIONAL,II CONCURSO DE FORTALECIMIENTO A LA CONTINUIDA...,R15A10002,2015,2016,LUIS ROJAS ARAYA,HOMBRE,BENEFICIARIA PRINCIPAL,CENTRO DE INVESTIGACION CIENTIFICO TECNOLOGICO...,SIN INFORMACION,CHILE,REGION DE ANTOFAGASTA,SCIA,CENTRO DE INVESTIGACION CIENTIFICO TECNOLOGICO...,65937660-1,True
2,PIA,CONCURSO NACIONAL DE ASIGNACION DE TIEMPO DE B...,AUB160003,2016,2016,RUBEN ESCRIBANO VELOSO,HOMBRE,INSTITUCION PATROCINANTE,UNIVERSIDAD DE CONCEPCION,81494400-K,CHILE,REGION DEL BIO-BIO,SCIA,UNIVERSIDAD DE CONCEPCION,81494400-k,True
3,REGIONAL,II CONCURSO DE FORTALECIMIENTO DE CENTROS REGI...,R16F10008,2016,2016,EDUARDO GRATACOS N.,HOMBRE,BENEFICIARIA PRINCIPAL,CENTRO REGIONAL DE INNOVACION HORTOFRUTICOLA ...,SIN INFORMACION,CHILE,REGION DE VALPARAISO,SCIA,CENTRO REGIONAL DE INNOVACION HORTOFRUTICOLA D...,-,False
4,REGIONAL,II CONCURSO DE FORTALECIMIENTO DE CENTROS REGI...,R16F10009,2016,2016,BERNARDO BROITMAN ROJAS,HOMBRE,BENEFICIARIA PRINCIPAL,CENTRO DE ESTUDIOS AVANZADOS EN ZONAS ARIDAS (...,SIN INFORMACION,CHILE,REGION DE COQUIMBO,SCIA,CENTRO DE ESTUDIOS AVANZADOS EN ZONAS ARIDAS -...,65997470-3,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23703,PCI,PROGRAMAS REGIONALES STIC-AMSUD - MATH-AMSUD -...,AMSUD220044,2022,2023,DENISSE MUÑANTE ARZAPALO,FEMENINO,INSTITUCION DE FRANCIA,ECOLE NATIONALE SUPERIEURE D’INFORMATIQUE POUR...,NaN,FRANCIA,NaN,SREC,ECOLE NATIONALE SUPERIEURE D'INFORMATIQUE POUR...,-,False
23704,PCI,PROGRAMAS REGIONALES STIC-AMSUD - MATH-AMSUD -...,AMSUD220045,2022,2023,HUGO CARRILLO LINCOPI,MASCULINO,INSTITUCION DE FRANCIA,INRIA PARIS,NaN,FRANCIA,NaN,SREC,INRIA PARIS,-,False
23705,PCI,PROGRAMAS REGIONALES STIC-AMSUD - MATH-AMSUD -...,AMSUD220045,2022,2023,HUGO CARRILLO LINCOPI,MASCULINO,INSTITUCION SUDAMERICANA,ESCUELA POLITECNICA NACIONAL,NaN,ECUADOR,NaN,SREC,ESCUELA POLITECNICA NACIONAL,-,False
23706,PCI,PROGRAMAS REGIONALES STIC-AMSUD - MATH-AMSUD -...,AMSUD220045,2022,2023,HUGO CARRILLO LINCOPI,MASCULINO,INSTITUCION SUDAMERICANA,FUNDACION INRIA CHILE,65058044-3,CHILE,NaN,SREC,FUNDACION INRIA CHILE,65058044-3,True


Veamos cuántas filas quedaron sin RUT.

In [ ]:
df_anid[(~df_anid["rut_valido"]) & (df_anid["PAISINSTITUCION"] == "CHILE")]

,PROGRAMA,CONCURSO,FOLIO,AGNOPOSTULACION,AGNOFALLO,RESPONSABLEPROYECTO,SEXO,ROLINSTITUCION,INSTITUCION1ERNIVEL,RUTINSTITUCION,PAISINSTITUCION,REGIONINSTITUCION,SUBDIRECCION,nombre_normalizado,rut_normalizado,rut_valido
0,REGIONAL,II CONCURSO DE FORTALECIMIENTO A LA CONTINUIDA...,R15A10001,2015,2016,EDUARDO GRATACOS,HOMBRE,BENEFICIARIA PRINCIPAL,CENTRO REGIONAL DE INNOVACION HORTOFRUTICOLA ...,SIN INFORMACION,CHILE,REGION DE VALPARAISO,SCIA,CENTRO REGIONAL DE INNOVACION HORTOFRUTICOLA D...,-,False
3,REGIONAL,II CONCURSO DE FORTALECIMIENTO DE CENTROS REGI...,R16F10008,2016,2016,EDUARDO GRATACOS N.,HOMBRE,BENEFICIARIA PRINCIPAL,CENTRO REGIONAL DE INNOVACION HORTOFRUTICOLA ...,SIN INFORMACION,CHILE,REGION DE VALPARAISO,SCIA,CENTRO REGIONAL DE INNOVACION HORTOFRUTICOLA D...,-,False
5,REGIONAL,II CONCURSO DE FORTALECIMIENTO DE CENTROS REGI...,R16F10002,2016,2016,CESAR BURGOS,HOMBRE,BENEFICIARIA PRINCIPAL,CENTRO DE GENOMICA NUTRICIONAL AGROACUICOLA (C...,SIN INFORMACION,CHILE,REGION DE LA ARAUCANIA,SCIA,CENTRO DE GENOMICA NUTRICIONAL AGROACUICOLA (C...,-,False
6,REGIONAL,II CONCURSO DE FORTALECIMIENTO DE CENTROS REGI...,R16F10013,2016,2016,DANIELA NUÃ±EZ BERNAL,MUJER,BENEFICIARIA PRINCIPAL,CENTRO DE INVESTIGACION EN POLIMEROS AVANZADOS...,SIN INFORMACION,CHILE,REGION DEL BIO-BIO,SCIA,CENTRO DE INVESTIGACION EN POLIMEROS AVANZADOS...,-,False
131,REGIONAL,IV CONCURSO DE FORTALECIMIENTO Y DESARROLLO DE...,R17A10002,2017,2017,ANA MOTRAN FERRANDIZ,MUJER,ENTIDAD ASOCIADA,INTENDENCIA REGIONAL DE AYSEN,SIN INFORMACION,CHILE,NO REGISTRA REGION,SCIA,INTENDENCIA REGIONAL DE AYSEN,-,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22954,PCI,FOMENTO A LA VINCULACION INTERNACIONAL PARA IN...,FOVI220124,2022,2022,MARISOL RUIZ CONTRERAS,MUJER,INSTITUCION ASOCIADA,ILUSTRE MUNICIPALIDAD DE VALDIVIA,-,CHILE,REGION DE LOS RIOS,SREC,ILUSTRE MUNICIPALIDAD DE VALDIVIA,-,False
22955,PCI,FOMENTO A LA VINCULACION INTERNACIONAL PARA IN...,FOVI220211,2022,2022,DANIEL MEDINA SALAS,HOMBRE,INSTITUCION ASOCIADA,SEREMI DE SALUD REGION DE LOS LAGOS,-,CHILE,NO REGISTRA REGION,SREC,SEREMI DE SALUD REGION DE LOS LAGOS,-,False
22956,PCI,FOMENTO A LA VINCULACION INTERNACIONAL PARA IN...,FOVI220211,2022,2022,DANIEL MEDINA SALAS,HOMBRE,INSTITUCION ASOCIADA,INSTITUTO NORUEGO DE INVESTIGACION DE AGUAS,-,CHILE,NO REGISTRA REGION,SREC,INSTITUTO NORUEGO DE INVESTIGACION DE AGUAS,-,False
23173,PCI,FOMENTO A LA VINCULACION INTERNACIONAL PARA IN...,FOVI220135,2022,2022,ARIEL LEIVA LOPEZ,HOMBRE,INSTITUCION EXTRANJERA,COMISION ECONOMICA PARA AMERICA LATINA Y EL CA...,NO APLICA,CHILE,REGION METROPOLITANA DE SANTIAGO,SREC,COMISION ECONOMICA PARA AMERICA LATINA Y EL CA...,-,False


Revisaremos por última vez que no haya ninguna institución chilena con más de un RUT asociado.

In [ ]:
for name in df_anid.loc[df_anid["PAISINSTITUCION"] == "CHILE", "nombre_normalizado"].unique():
  ruts = df_anid.loc[df_anid["nombre_normalizado"] == name, "rut_normalizado"].unique()
  if len(ruts) != 1:
    print(name, ruts, len(ruts))

Todo bien. Guardaremos el dataframe.

In [ ]:
df_anid.to_excel(os.path.join(results_dir, "data_anid_ruts_imputados.xlsx"), index=False)